In [ ]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Done.")

<IPython.core.display.Javascript object>

Done.


##Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


##Libraries

####Install


In [ ]:
pip install keras-self-attention

  Created wheel for keras-self-attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18912 sha256=477b98c53503d102171ac0a482e4f460e601ffaa22b0cf31bf844f6b004adb87
  Stored in directory: /root/.cache/pip/wheels/95/b1/a8/5ee00cc137940b2f6fa198212e8f45d813d0e0d9c3a04035a3
Successfully built keras-self-attention


####Import

In [ ]:
# For data processing
import numpy as np
import math
from math import sqrt

# For data processing and manipulation
import pandas as pd
import csv

# For date calculations
import datetime
import time

# For ploting data
import IPython
import IPython.display

import itertools
from itertools import cycle
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# For checking path
import os , gc
from os import path
import csv
import json


from scipy.stats import hmean

from sklearn.metrics import mean_squared_error

#tensorflow libs
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping , Callback
import tensorflow as tf
from tensorflow.keras import backend as K
from keras import backend

from tensorflow.keras.layers import *
from tensorflow.keras.layers import Dense , LSTM ,Dropout , PReLU , RepeatVector ,TimeDistributed, Attention
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical , plot_model
from tensorflow.keras import regularizers, constraints, initializers, activations
#from keras.layers.recurrent import Recurrent, _time_distributed_dense
from tensorflow.keras.layers import SimpleRNN as Recurrent
#from tensorflow.compat.v1.keras.layers import RNN 

from tensorflow.keras.layers import InputSpec

from keras_self_attention import SeqSelfAttention
from keras.layers.merge import concatenate


tf.get_logger().setLevel('ERROR')
mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [ ]:
print(tf.__version__)

2.8.0


In [ ]:
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']

##Root Path

In [ ]:
# ROOTPATH = '/content/gdrive/MyDrive/Projects/Crime forcasting'
ROOTPATH = '/content/gdrive/MyDrive/Colab Notebooks/ThesisFinal'
PATH_IMAGE = ROOTPATH+'/images_chicago_attention/'
PATH_DA = ROOTPATH+"/datasets/chicago-crime/chicago_data_sf_2004_2017_for_da.csv"
PATH_MAIN = ROOTPATH + "/datasets/chicago-crime/Crimes_-_2001_to_Present.csv"
PATH_PDA = ROOTPATH+"/datasets/chicago-crime/chicago_data_sf_2004_2017_for_police_da.csv"
PATH_DA_CAT = ROOTPATH+"/datasets/chicago-crime/chicago_data_sf_2004_2017_for_da_cat.csv"
PATH_WEATHER = ROOTPATH+"/datasets/chicago-crime/weather_data_chicago_2004_2017.csv"
PATH_TEMP = ROOTPATH+"/datasets/chicago-crime/temp_data_chicago_2004_2017.csv"
date_after = pd.to_datetime("1/1/2004  00:00:00 AM")
date_before = pd.to_datetime("1/1/2018  00:00:00 AM")

##Data Pre-Processing

####Load the data

In [ ]:
# tf.debugging.set_log_device_placement(False)
gpus = tf.config.list_logical_devices('CPU')
strategy = tf.distribute.MirroredStrategy(gpus)

In [ ]:
with strategy.scope():
    dataset = pd.read_csv(PATH_DA)
    dataset.head()

####Date Time

The data from 2004 to 2017 are taken

In [ ]:
date_after = pd.to_datetime("1/1/2004  00:00:00 AM")
date_before = pd.to_datetime("1/1/2018  00:00:00 AM")


In [ ]:
# dataset['datetime'] = pd.to_datetime(dataset['Date']+' '+dataset['Time'])
# dataset = dataset[(dataset.datetime>=date_after) & (dataset.datetime<date_before)]
# dataset = dataset.sort_values(by = ['datetime'])

# dataset = dataset[["datetime","Date","Category","PdDistrict"]]

# dataset.to_csv(ROOTPATH+'/datasets/sf-crime/sf_crime_data_2004_2017.csv',index=False)

In [ ]:
# dataset = pd.read_csv(ROOTPATH+'/datasets/sf-crime/sf_crime_data_2004_2017.csv')

In [ ]:
dataset.shape

(5071819, 12)

####PdDistrict

In [ ]:
dataset['PdDistrict'].isnull().sum()

0

In [ ]:
if(dataset['PdDistrict'].isnull().sum()):
    dataset = dataset.drop( index= dataset.loc[dataset['PdDistrict'].isnull()].index[0]) 

In [ ]:
dataset['PdDistrict'].isnull().sum()

0

In [ ]:
dataset = dataset.reset_index(drop= True)
dataset['datetime'] = dataset['Date']

In [ ]:
num_police_dept = dataset['PdDistrict'].nunique()
print("Police departement district = ",num_police_dept)
Police_dept_name = dataset['PdDistrict'].drop_duplicates().sort_values(ascending = True)
print(Police_dept_name)

Police departement district =  24
0         1.0
33        2.0
4         3.0
8         4.0
15        5.0
3         6.0
20        7.0
5         8.0
2         9.0
6        10.0
21       11.0
54       12.0
7        14.0
9        15.0
23       16.0
17       17.0
40       18.0
29       19.0
1        20.0
55159    21.0
113      22.0
12       24.0
11       25.0
13211    31.0
Name: PdDistrict, dtype: float64


In [ ]:
with strategy.scope():
    # Dictionary for mapping id to name of the unique police department
    Police_dept_id_name = {i:Police_dept_name.values[i] for i in range (num_police_dept)}
    print("Police Departments ID:Name Dict-> \n",Police_dept_id_name)

    # Dictionary for mapping name of the unique police department to id
    Police_dept_name_id = {key:value for (value,key) in Police_dept_id_name.items()}
    print("Police Departments Name:ID Dict-> \n",Police_dept_name_id)

    # Update the new column of dataframe with the value of the list 
    dataset["PdDID"] = dataset['PdDistrict']
    print("PdDID column -> \n", dataset["PdDID"])

Police Departments ID:Name Dict-> 
 {0: 1.0, 1: 2.0, 2: 3.0, 3: 4.0, 4: 5.0, 5: 6.0, 6: 7.0, 7: 8.0, 8: 9.0, 9: 10.0, 10: 11.0, 11: 12.0, 12: 14.0, 13: 15.0, 14: 16.0, 15: 17.0, 16: 18.0, 17: 19.0, 18: 20.0, 19: 21.0, 20: 22.0, 21: 24.0, 22: 25.0, 23: 31.0}
Police Departments Name:ID Dict-> 
 {1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3, 5.0: 4, 6.0: 5, 7.0: 6, 8.0: 7, 9.0: 8, 10.0: 9, 11.0: 10, 12.0: 11, 14.0: 12, 15.0: 13, 16.0: 14, 17.0: 15, 18.0: 16, 19.0: 17, 20.0: 18, 21.0: 19, 22.0: 20, 24.0: 21, 25.0: 22, 31.0: 23}
PdDID column -> 
 0           1.0
1          20.0
2           9.0
3           6.0
4           3.0
           ... 
5071814     5.0
5071815    12.0
5071816    18.0
5071817    19.0
5071818    17.0
Name: PdDID, Length: 5071819, dtype: float64


In [ ]:
with strategy.scope():
    dataset.to_csv(PATH_PDA,index=False)

######Time series based

In [ ]:
with strategy.scope():
    dataset1 = pd.read_csv(PATH_PDA)

In [ ]:
with strategy.scope():
    print(dataset1.shape)

(5071819, 13)


In [ ]:
def processCategoryPd(df):
    for col in df.columns:
        col = str(col)
        if(col=='Category'):
            continue
        if col=='datetime':
            continue
        if col=='PdDID':
            continue
        if col=='PdDistrict':
            continue
        df.drop(col , 1 , inplace=True)

    Category = np.array(df['Category'].unique() , str)
    Category.sort()
    Category = Category.tolist()
    #print(Category)
    df['Category'] = df['Category'].apply(lambda x:float(Category.index(x)))
    Y = pd.DataFrame(to_categorical(df['Category']))
    df = df.join(Y , on=Y.index)
    df['count'] = df['Category'].apply(lambda x:float(1))
    df.drop('Category' , 1, inplace=True)
    df.set_index("datetime" , inplace = True)
    df.index =  pd.to_datetime(df.index)
    #df_sample = df.resample(sample).sum()
    return df, Category

In [ ]:
with strategy.scope():
    df1 = processCategoryPd(dataset1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


In [ ]:
with strategy.scope():
    df1 , Category1 = df1
    print(df1.shape)
    df1.tail()

(5071819, 34)


In [ ]:
with strategy.scope():
    all_datasets = {}
    new_Police_dept_name = [] #dataset['PdDistrict'].drop_duplicates().sort_values(ascending = True)
    for x in Police_dept_name:
        tmp = df1[(df1.PdDistrict == x)]
        if(tmp.shape[0]<200):
            # print("kom ",  x , tmp.shape)
            continue
        new_Police_dept_name.append(x)

        all_datasets[x] = tmp
        # print(all_datasets[x].shape)
    Police_dept_name = new_Police_dept_name

(198647, 34)
(222946, 34)
(261296, 34)
(293522, 34)
(227875, 34)
(298434, 34)
(303855, 34)
(350117, 34)
(251525, 34)
(219618, 34)
(327624, 34)
(244782, 34)
(192801, 34)
(224142, 34)
(168056, 34)
(146331, 34)
(217509, 34)
(223202, 34)
(86015, 34)
kom  21.0 (3, 34)
(168455, 34)
(150591, 34)
(294309, 34)
kom  31.0 (164, 34)


In [ ]:
with strategy.scope():
    dataset2 = pd.read_csv(PATH_PDA)

In [ ]:
def processCategory(df):
    for col in df.columns:
        col = str(col)
        if(col=='Category'):
            continue
        if col=='datetime':
            continue
        df.drop(col , 1 , inplace=True)

    Category = np.array(df['Category'].unique() , str)
    Category.sort()
    Category = Category.tolist()
    df['Category'] = df['Category'].apply(lambda x:float(Category.index(x)))
    Y = pd.DataFrame(to_categorical(df['Category']))
    df = df.join(Y , on=Y.index)
    df['count'] = df['Category'].apply(lambda x:float(1))
    df.drop('Category' , 1, inplace=True)


    df.set_index("datetime" , inplace = True)
    df.index =  pd.to_datetime(df.index)
    return df, Category

In [ ]:
with strategy.scope():
    df2 = processCategory(dataset2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


In [ ]:
with strategy.scope():
    df2 , Category2 = df2
    df2.head()

In [ ]:
with strategy.scope():
    print(df2.shape)
    df2.tail()

(5071819, 32)


####Temperature

In [ ]:
with strategy.scope():
    weather_dataset = pd.read_csv(PATH_WEATHER)

In [ ]:
with strategy.scope():
    temperature_dataset = pd.DataFrame(weather_dataset[["valid_date_time", "temp"]].copy())
    temperature_dataset = temperature_dataset.rename(columns={"valid_date_time": "datetime","temp" : "temperature"})
    temperature_dataset["datetime"] = pd.to_datetime(temperature_dataset["datetime"])
    temperature_dataset = temperature_dataset.dropna()
    temperature_dataset = temperature_dataset[(temperature_dataset.datetime>=date_after) & (temperature_dataset.datetime<date_before)]
    temperature_dataset = temperature_dataset.drop_duplicates(subset =["datetime"])
    temperature_dataset.shape

In [ ]:
with strategy.scope():
    temperature_dataset["temperature"].isnull().sum()

In [ ]:
with strategy.scope():
    temperature_dataset.to_csv(PATH_TEMP,index = False)

In [ ]:
with strategy.scope():
    temperature_dataset["temperature"].max()

In [ ]:
with strategy.scope():
    temperature_dataset["temperature"].min()

####Categorical

In [ ]:
def processData(df,sample):
    '''
    Change the sorted crime category with their index value
    & Resample Date time using hour
    input:
        df  : Dataset to process
        sample  : How to resample dataset df
    output:
        df_sample : resampled dataset df
    '''

    df_sample = df.resample(sample).sum()
    return df_sample

In [ ]:
with strategy.scope():
    for x in Police_dept_name:
        all_datasets[x] = processData(all_datasets[x] , '1H')
        print(x , all_datasets[x].shape)

1.0 (122735, 34)
2.0 (122735, 34)
3.0 (122736, 34)
4.0 (122736, 34)
5.0 (122736, 34)
6.0 (122736, 34)
7.0 (122736, 34)
8.0 (122736, 34)
9.0 (122735, 34)
10.0 (122736, 34)
11.0 (122736, 34)
12.0 (122736, 34)
14.0 (122735, 34)
15.0 (122735, 34)
16.0 (122736, 34)
17.0 (122736, 34)
18.0 (122736, 34)
19.0 (122736, 34)
20.0 (122736, 34)
22.0 (122736, 34)
24.0 (122736, 34)
25.0 (122735, 34)


In [ ]:
with strategy.scope():
    for x in Police_dept_name:
        all_datasets[x]['PdDID']= Police_dept_name_id[x]
        all_datasets[x] = pd.merge(all_datasets[x] , temperature_dataset , on='datetime')
        all_datasets[x].set_index('datetime' , inplace=True)
        all_datasets[x].index = pd.to_datetime(all_datasets[x].index)

In [ ]:
with strategy.scope():
    for x in Police_dept_name:
        print(all_datasets[x].shape)

(121620, 35)
(121620, 35)
(121621, 35)
(121621, 35)
(121621, 35)
(121621, 35)
(121621, 35)
(121621, 35)
(121620, 35)
(121621, 35)
(121621, 35)
(121621, 35)
(121620, 35)
(121620, 35)
(121621, 35)
(121621, 35)
(121621, 35)
(121621, 35)
(121621, 35)
(121621, 35)
(121621, 35)
(121620, 35)


In [ ]:
with strategy.scope():
    df2 = processData(df2 , '1H')
    df2.index = pd.to_datetime(df2.index)
    df2 = pd.merge(df2 , temperature_dataset , on='datetime')
    df2.set_index('datetime' , inplace=True)
    df2.index = pd.to_datetime(df2.index)

In [ ]:
with strategy.scope():
    df2.head()

In [ ]:
with strategy.scope():
    print(df2.shape)

(121621, 33)


####Time to signal

As we are trying to work with the periodical nature of crime, we are using encoded cyclic feature.
 

In [ ]:
def dateTimeToSignal(df):
    '''
    Converts the DateTime index to timestamp and convert it to signal (sine and cosine) to deal with periodicity
    input: 
        df : Dataset
    Output:
        df : Dataset with column Day sin , Day cos , Week sin , Week cos ,  Year sin , Year cos ; representing Sin / Cosine signal for timestamp  
    '''
    timestamp_s = df.index.map(datetime.datetime.timestamp)
    day = 24*60*60
    week = 7*day
    year = (365.2425)*day

    df['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
    df['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))

    df['Week sin'] = np.sin(timestamp_s * (2 * np.pi / week))
    df['Week cos'] = np.cos(timestamp_s * (2 * np.pi / week))

    df['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
    df['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
    return df

In [ ]:
with strategy.scope():
    for x in Police_dept_name:
        all_datasets[x] = dateTimeToSignal(all_datasets[x])

In [ ]:
with strategy.scope():
    df2 = dateTimeToSignal(df2)

In [ ]:
with strategy.scope():
    mini = 1000000000
    common_date = None
    for x in Police_dept_name:
        # print(all_datasets[x].shape)
        if all_datasets[x].shape[0] < mini:
            mini = all_datasets[x].shape[0];
            common_date = all_datasets[x].index

    for x in Police_dept_name:
        if(all_datasets[x].shape[0]>mini):
            delItems = []
            for i in all_datasets[x].index:
                if(i not in common_date):
                    delItems.append(i)
                    # print(i)
            all_datasets[x].drop(delItems , inplace=True)
        # print(x , all_datasets[x].shape)

1.0 (121620, 41)
2.0 (121620, 41)
2017-12-31 23:00:00
3.0 (121620, 41)
2017-12-31 23:00:00
4.0 (121620, 41)
2017-12-31 23:00:00
5.0 (121620, 41)
2017-12-31 23:00:00
6.0 (121620, 41)
2017-12-31 23:00:00
7.0 (121620, 41)
2017-12-31 23:00:00
8.0 (121620, 41)
9.0 (121620, 41)
2017-12-31 23:00:00
10.0 (121620, 41)
2017-12-31 23:00:00
11.0 (121620, 41)
2017-12-31 23:00:00
12.0 (121620, 41)
14.0 (121620, 41)
15.0 (121620, 41)
2017-12-31 23:00:00
16.0 (121620, 41)
2017-12-31 23:00:00
17.0 (121620, 41)
2017-12-31 23:00:00
18.0 (121620, 41)
2017-12-31 23:00:00
19.0 (121620, 41)
2017-12-31 23:00:00
20.0 (121620, 41)
2017-12-31 23:00:00
22.0 (121620, 41)
2017-12-31 23:00:00
24.0 (121620, 41)
25.0 (121620, 41)


In [ ]:
# date_all = df2.index
# date_range = pd.date_range(date_after,date_before)
# for x in Police_dept_name:
#   if(len(all_datasets[x])<len(date_all)):
#     for i in date_all:
#       if(i not in all_datasets[x].index):
#         all_datasets[x].loc[i] = all_datasets[x].loc[date_after]

In [ ]:
with strategy.scope():
    for x in Police_dept_name:
        print(x , all_datasets[x].shape)

1.0 (121620, 41)
2.0 (121620, 41)
3.0 (121620, 41)
4.0 (121620, 41)
5.0 (121620, 41)
6.0 (121620, 41)
7.0 (121620, 41)
8.0 (121620, 41)
9.0 (121620, 41)
10.0 (121620, 41)
11.0 (121620, 41)
12.0 (121620, 41)
14.0 (121620, 41)
15.0 (121620, 41)
16.0 (121620, 41)
17.0 (121620, 41)
18.0 (121620, 41)
19.0 (121620, 41)
20.0 (121620, 41)
22.0 (121620, 41)
24.0 (121620, 41)
25.0 (121620, 41)


####Group Distribution

In the Dataset, there are many categories whose count is less then 100000 and the lowest count is 14. There are only 7 categories having count greater than 100000. So, we've taken this 7 categories as they are and then made 3 groups with rest of the categories.   

In [ ]:
with strategy.scope():
    # Group define
    GROUPS = [[15],
        [2],
        [5],
        [8],
        [1],
        [21],
        [3],
        [29,7],
        [26,6],
        [30,22,24,19,27,12,9,16,0,10,14,13,28,18,17,4,23,20,11,25]]

In [ ]:
def make_groups(df):
    '''
    Make 10 groups from 38 different type of crime categories. 
    Where,
      GR0 consits of,
        29 	VEHICLE THEFT 
        7 	DECEPTIVE PRACTICE
      GR1 consits of,
        26 	ROBBERY 
        6 	CRIMINAL TRESPASS
      GR2 consits of,
        30 	WEAPONS VIOLATION 		
        22 	PROSTITUTION 				
        24 	PUBLIC PEACE VIOLATION 		
        19 	OFFENSE INVOLVING CHILDREN 	
        27 	SEX OFFENSE 			
        12 	INTERFERENCE WITH PUBLIC OFFICER 	
        9 	GAMBLING 				
        16 	LIQUOR LAW VIOLATION 			
        0 	ARSON 				  	
        10 	HOMICIDE 				
        14 	KIDNAPPING 				
        13 	INTIMIDATION 				
        28 	STALKING 				
        18 	OBSCENITY 				 
        17 	NON-CRIMINAL 				 
        4 	CONCEALED CARRY LICENSE VIOLATION 	 
        23 	PUBLIC INDECENCY 			 
        20 	OTHER NARCOTIC VIOLATION 		  
        11 	HUMAN TRAFFICKING 			  
        25 	RITUALISM 

    input:
        df : Dataset with count of individual's from 38 categories
    output:
        df : Dataset with count of 10 groups  
    '''
    cnt = 0
    for i in GROUPS:
        cols_to_sum = i
        if(len(i)==1):
            continue
        else:
            newname = "GRP"+str(cnt)
            cnt = cnt + 1
            try:
                df[newname] = df[cols_to_sum].sum(axis=1)
                for j in i:
                    df.drop(j , 1 , inplace=True)
            except: 
                continue
    return df


In [ ]:
with strategy.scope():
    for x in Police_dept_name:
        print(x)
        all_datasets[x] = make_groups(all_datasets[x])
        all_datasets[x] = all_datasets[x].rename(columns={15:1,2: 2, 5 : 3,8 : 4,1 : 5,21 : 6,3 : 7})

1.0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
22.0
24.0
25.0


In [ ]:
with strategy.scope():
    df2 = make_groups(df2) 
    df2 = df2.rename(columns={15:1,
                            2: 2,
                            5 : 3,
                            8 : 4,
                            1 : 5,
                            21 : 6,
                            3 : 7})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


In [ ]:
with strategy.scope():
    names = {}
    names['1'] = Category1[15]
    names['2'] = Category1[2]
    names['3'] = Category1[5]
    names['4'] = Category1[8]
    names['5'] = Category1[1]
    names['6'] = Category1[21]
    names['7'] = Category1[3]
    names['count'] = 'COUNT'
    names['GRP0'] = 'GRP0'
    names['GRP1'] = 'GRP1'
    names['GRP2'] = 'GRP2'

In [ ]:
with strategy.scope():
    df2.index = pd.to_datetime(df2.index)
    df2.head()

####Temperature categorized

In [ ]:
def temp_scale(x):
    '''
    x <= 273K or 0C -> low(0) 
    x > 303K or 30C-> high(2)
    else medium(1)
    '''
    if(x<=0):
        return 0
    elif(x>30):
        return 2
    else:
        return 1

In [ ]:
with strategy.scope():
    for x in Police_dept_name:
        all_datasets[x]['temperature'] = all_datasets[x]['temperature'].apply(lambda x:temp_scale(x))

##Some functions

In [ ]:
def split_data(df):
    '''
    split the dataset ( train , val , test = (70 , 20 , 10)% )
    input:
        df : Dataset to split
    output:
        train_df : train dataset
        val_df : validation dataset
        test_df : test dataset
        num_features : Number of features in dataset
        column_indices : column_indices in dataset
    '''
    column_indices = {name: i for i, name in enumerate(df.columns)}

    n = len(df)
    train_df = df[0:int(n*0.7)]
    val_df = df[int(n*0.7):int(n*0.9)]
    test_df = df[int(n*0.9):]

    num_features = df.shape[1]
    return train_df , val_df , test_df , num_features , column_indices

In [ ]:
#@title
class WindowGenerator():
    '''
    WindowGenerator Class
    1. Split windows of features into a (features, labels) pairs.
    2. Plot the content of the resulting windows.
    3. Efficiently generate batches of these windows from the training, evaluation, and test data, using tf.data.Datasets S.
    '''
    def __init__(self, input_width, label_width, shift,
               train_df, val_df, test_df,
               label_columns=None , shuffle=False , batch_size = 64):
        '''
        The __init__ method includes all the necessary logic for the input and label indices.
        Input: 
            input_width : input width / window size 
            label_width : output width
            shift : size of window shifting forward
            train_df : train dataset
            val_df : validation dataset
            test_df : test dataset
            label_columns ( Default = None) : Label Columns
            shuffle ( Default = False) : weather to shuffle data 
            batch_size (Default = 64) : Batch Size
        Output: None
        Example :
            w2 = WindowGenerator(input_width=6, label_width=1, shift=1,
                     label_columns=['count'])
            w2

        '''
        # Store the raw data.
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df
        self.shuffle = shuffle
        self.batch_size = batch_size

        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in
                                        enumerate(label_columns)}
            
        self.column_indices = {name: i for i, name in
                            enumerate(train_df.columns)}


        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width) #(start , stop)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])
        
    def split_window(self, features):
        '''
        Given a list consecutive inputs, the split_window method will convert them to a window of inputs and a window of labels.
        
        Input: 
            features : Stack of array of datas , used for splitting data to inputs and labels
        Output:
            inputs : nd Array splitted as input_width
            labesl : nd Array splitted as label_width
        Example :
        # Stack three slices, the length of the total window:
            example_window = tf.stack([np.array(train_df[:w2.total_window_size]),
                           np.array(train_df[100:100+w2.total_window_size]),
                           np.array(train_df[200:200+w2.total_window_size])])


            example_inputs, example_labels = w2.split_window(example_window)

            print('All shapes are: (batch, time, features)')
            print(f'Window shape: {example_window.shape}')
            print(f'Inputs shape: {example_inputs.shape}')
            print(f'labels shape: {example_labels.shape}')
        '''
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        #taking only the labels that are presentin the label_columns
        if self.label_columns is not None:
            labels = tf.stack([labels[:, :, self.column_indices[name]] for name in self.label_columns],axis=-1)

        # Slicing doesn't preserve static shape information, so set the shapes
        # manually. This way the `tf.data.Datasets` are easier to inspect.
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels

    def plot(self, model=None, plot_col='count', max_subplots=3):
        '''
            plot method that allows a simple visualization of the split window,
            Input:
                model (Default=None) : tensorflow model to evaluate 
                plot_col ( Default = 'count') : Name of column to evaluate
                max_subplots ( Default = 3) : Maximum Number of subplotting
            Output:
                None
            Example:
                w2.plot()
                w2.plot(plot_col=0) # label wont be shown as w2 config has only column , count 

        '''
        inputs, labels = self.example
        plt.figure(figsize=(12, 8))
        plot_col_index = self.column_indices[plot_col]
        max_n = min(max_subplots, len(inputs))
        for n in range(max_n):
            plt.subplot(max_n, 1, n+1)
            plt.ylabel(f'{plot_col} [normed]')
            plt.plot(self.input_indices, inputs[n, :, plot_col_index],label='Inputs', marker='.', zorder=-10)

            if self.label_columns:
                label_col_index = self.label_columns_indices.get(plot_col, None)
            else:
                label_col_index = plot_col_index

            if label_col_index is None:
                continue

            plt.scatter(self.label_indices, labels[n, :, label_col_index],edgecolors='k', label='Labels', c='#2ca02c', s=64)
            if model is not None:
                predictions = model(inputs)
                plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                        marker='X', edgecolors='k', label='Predictions',c='#ff7f0e', s=64)
            if n == 0:
                plt.legend()
        plt.xlabel('Time [h]')

    def make_dataset(self, data):
        '''
        make_dataset method will take a time series DataFrame and convert it to a
            tf.data.Dataset of (input_window, label_window) pairs using the preprocessing.timeseries_dataset_from_array function.
        Input:
            data :  Input data to transform into (input_window , label_window)
        Output:
            ds : transformed dataset
        '''
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.preprocessing.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=self.shuffle,
            batch_size=self.batch_size,)
        ds = ds.map(self.split_window)
        return ds
    
    
    def create_dataset2(self , map_df , reshape=True):
      x = []
      y = []
      for res in iter(map_df):
        inputs, labels = res
        if(len(inputs)==64):
          x.append(inputs)
          y.append(labels)
      
      x = np.array(x)
      y = np.array(y)
      if(reshape):
        x = x.reshape(-1, x.shape[-2] , x.shape[-1])
        y = y.reshape(-1 , y.shape[-2] , y.shape[-1])
      return x , y
    
    '''
    properties for accessing  training, validation and test data as tf.data.Datasets using the above make_dataset method.
    Also a standard example batch for easy access and plotting
    '''

    @property
    def train(self):
        return self.make_dataset(self.train_df)

    @property
    def val(self):
        return self.make_dataset(self.val_df)

    @property
    def test(self):
        return self.make_dataset(self.test_df)
    
    
    @property
    def example(self):
        """Get and cache an example batch of `inputs, labels` for plotting."""
        result = getattr(self, '_example', None)
        if result is None:
            # No example batch was found, so get one from the `.train` dataset
            result = next(iter(self.train))
            # And cache it for next time
            self._example = result
        return result

In [ ]:
def create_data(train , test , val , columns):
    '''
    Create dataset from main train , test , val with given columns
    '''
    if(columns==None):
        columns = train.columns
    new_train = train[columns]
    new_test = test[columns]
    new_val = val[columns]
    return new_train , new_test , new_val

In [ ]:
def save_history(history , path):
    # convert the history.history dict to a pandas DataFrame:     
    hist_df = pd.DataFrame(history.history) 
    # or save to csv: 
    hist_csv_file = path
    with open(hist_csv_file, mode='w') as f:
        hist_df.to_json(f)

def get_history(path):
	with open(path) as json_file:
		data = json.load(json_file)
		return data

def save_model_weights(model , path):
  model.save_weights(path)


In [ ]:
MAX_EPOCHS = 20

def compile_and_fit(model, modelPath , historyPath , window=None, name=None , patience=10):
    '''
    Compile and fit a model
    '''
    modelPathPar = ROOTPATH+'/'+modelPath+'/'
    historyPathPar = ROOTPATH+'/'+historyPath+'/'
    if (name!=None):
        # model_path = ROOTPATH+'/model_all_mae_swish/'+name
        # history_path = ROOTPATH+'/history_all_mae_swish/'+name+".json"
        model_path = modelPathPar+name
        history_path = historyPathPar+name+".json"
        
        if not path.exists(modelPathPar):
            os.makedirs(modelPathPar)
        if not path.exists(historyPathPar):
            os.makedirs(historyPathPar)

    if (name!=None and path.exists(model_path)):
      print("Loaded Pre Trained Model")
      modelOld = tf.keras.models.load_model(model_path)
      model.set_weights(modelOld.get_weights())
      del modelOld
      history = get_history(history_path)
      return model , history
      
    
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=patience)
    
    history = model.fit(window.train, epochs=MAX_EPOCHS,validation_data=window.val,callbacks=[early_stopping])
    if(name!=None):
      model.save(model_path)
      save_history(history , history_path)
    return model , history.history


In [ ]:
def sMAPE(y, y_pred):
    epsilon = 0.1
    summ = backend.maximum(backend.abs(y) + backend.abs(y_pred) + epsilon, 0.5 + epsilon)
    smape = backend.abs(y_pred - y) / summ * 2.0
    return smape

In [ ]:
def R_squared(y, y_pred):
  residual = tf.reduce_sum(tf.square(tf.subtract(y, y_pred)))
  total = tf.reduce_sum(tf.square(tf.subtract(y, tf.reduce_mean(y))))
  r2 = tf.subtract(1.0, tf.divide(residual, total))
  return r2

In [ ]:
MAX_EPOCHS = 200
plt.rcParams.update({'font.size': 30})
metrics_name = ["MAE" , "MSLE" , "MSE" , "SMAPE" , "R^2"]

def compileModel(model):
    with strategy.scope():
        model.compile(loss=tf.losses.MeanAbsoluteError(),optimizer=tf.optimizers.Adam(),
                    metrics=[tf.metrics.MeanSquaredLogarithmicError() , tf.metrics.MeanSquaredError() , sMAPE , R_squared ])
        return model   
    
def compile_and_fit_merged(model, train_1, train_2, val_1, val_2, train_y , val_y, name=None , patience=10):
    '''
    Compile and fit a model
    '''
    with strategy.scope():
        if(name!=None):
            model_path = ROOTPATH+'/model_chicago_merge/'+name
            history_path = ROOTPATH+'/history_chicago_merge/'+name+".json"
            model_plot_path = ROOTPATH+'/model_plot_merge/'+name+".png"
            checkpoint_path = ROOTPATH+"/checkpoint_chicago_merge/"+name+".ckpt"
        

            #plot_model(model, model_plot_path, show_shapes=True , expand_nested=True)

        if not path.exists(ROOTPATH+'/model_chicago_merge/'):
            os.makedirs(ROOTPATH+'/model_chicago_merge/')
        if not path.exists(ROOTPATH+'/history_chicago_merge/'):
            os.makedirs(ROOTPATH+'/history_chicago_merge/')
        if not path.exists(ROOTPATH+'/model_plot_merge/'):
            os.makedirs(ROOTPATH + '/model_plot_merge/')
        if not path.exists(ROOTPATH+"/checkpoint_chicago_merge/"):
            os.makedirs(ROOTPATH+"/checkpoint_chicago_merge/")

        
        if(name!=None and path.exists(model_path)):
            print("Loaded Pre Trained Model")
            modelOld = tf.keras.models.load_model(model_path ,  custom_objects={'sMAPE':sMAPE , "R_squared":R_squared})
            model.set_weights(modelOld.get_weights())
            del modelOld
            history = get_history(history_path)
            return model , history
            
        if(name!=None and path.exists(ROOTPATH+"/checkpoint_chicago_merge/")):
            print("loaded weights")
            try:
                model.load_weights(checkpoint_path)
            except tf.errors.NotFoundError:
                print("not found")

        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=patience , restore_best_weights = True)

        checkpoint_dir = os.path.dirname(checkpoint_path)
        # print(checkpoint_dir)

        # Create a callback that saves the model's weights
        cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                    save_weights_only=True,
                                                    verbose=1)
        history = model.fit([train_1,train_2], train_y, batch_size=64, epochs=MAX_EPOCHS, validation_data=([val_1,val_2],val_y), callbacks=[early_stopping , cp_callback])
        if(name!=None):
            model.save(model_path)
            save_history(history , history_path)
        return model , history.history

input-> features = [time_signal , count_of_last_24h ]

prediction = [next_hour_count]

In [ ]:
general_indexs = ['PdDID' , 1,2,3,4,5,6,7,'count','temperature','Day sin','Day cos','Week sin','Week cos','Year sin','Year cos','GRP0','GRP1','GRP2']
x_col = ['Day sin' , 'Day cos' , 'Year sin' , 'Year cos' , 'Week cos' , 'Week sin' ,'temperature' , 'datetime' , 'PdDID']

def generate_window(df_now, ret_test = 0):
    train_df , val_df , test_df , num_features_df , column_indices_df = split_data(df_now)
    train_df , test_df , val_df = create_data(train_df , test_df , val_df , general_indexs)
    y_col = [] 

    for i in train_df.columns:
        if (i in x_col):
            continue
        y_col.append(i)
    wide_window_all = WindowGenerator(train_df=train_df, test_df=test_df , val_df=val_df, 
        input_width=24, label_width=24, shift=1,
        label_columns=y_col)
    
    if (ret_test == 1):
      return wide_window_all, test_df
    else:  
      return wide_window_all   

In [ ]:
general_indexs2 = [ 1,2,3,4,5,6,7,'count', 'Day sin' , 'Day cos' , 'Week cos' , 'Week sin', 'Year sin' , 'Year cos' , 'GRP0','GRP1','GRP2']

x_col2 = ['Day sin' , 'Day cos' , 'Year sin' , 'Year cos' , 'Week cos' , 'Week sin' , 'datetime']

def generate_window2(df_now):
    train_df , val_df , test_df , num_features_df , column_indices_df = split_data(df_now)
    train_df , test_df , val_df = create_data(train_df , test_df , val_df , general_indexs2)
    
    y_col = []
    for i in train_df.columns:
        if(i in x_col2):
            continue
        y_col.append(i)
    wide_window_all = WindowGenerator(train_df=train_df, test_df=test_df , val_df=val_df, input_width=24, label_width=24, shift=1, label_columns=y_col)
    
    return wide_window_all    

In [ ]:
import json 
from json import JSONEncoder
class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)
def save_json(filename , data):
    with open(filename, 'w') as fp:
        json.dump(data, fp, cls=NumpyArrayEncoder)
        fp.close()
def read_json(filename):
    with open(filename) as fp:
        dictdump = json.loads(fp.read())
        fp.close()
        return dictdump


In [ ]:
def print_perf_val_perf(perf , val_perf):
    print(f'{"model":50s}: Test   | Val')
    for i in range(len(metrics_name)):
        print("==================",metrics_name[i],"==================")
        for key in val_perf.keys():
            print(f'{str(key):50s}: {perf[key][i]:0.4f} | {val_perf[key][i]:0.4f}')

##Models

In [ ]:
with strategy.scope():
    input_bi_lstm = keras.Input(shape=(24,19), name="input_BiLSTM")
    x_bd = (keras.layers.Bidirectional(tf.keras.layers.LSTM(128, activation = tf.keras.activations.swish ,return_sequences=True) , name="bilstm1"))(input_bi_lstm)
    x_bd = (keras.layers.Bidirectional(tf.keras.layers.LSTM(64, activation = tf.keras.activations.swish ,return_sequences=True) , name="bilstm2"))(x_bd)
    x_bd = (TimeDistributed(keras.layers.Dense(units=64, activation = tf.keras.activations.swish), name="TimeDisDenseLSTM"))(x_bd)
    output_bi_lstm = (keras.layers.Dense(units=11))(x_bd)
    
    bi_lstm_model_all_feature = keras.Model(
        inputs=[input_bi_lstm],
        outputs=[output_bi_lstm],
    )
    bi_lstm_model_all_feature.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_BiLSTM (InputLayer)   [(None, 24, 19)]          0         
                                                                 
 bilstm1 (Bidirectional)     (None, 24, 256)           151552    
                                                                 
 bilstm2 (Bidirectional)     (None, 24, 128)           164352    
                                                                 
 TimeDisDenseLSTM (TimeDistr  (None, 24, 64)           8256      
 ibuted)                                                         
                                                                 
 dense_1 (Dense)             (None, 24, 11)            715       
                                                                 
Total params: 324,875
Trainable params: 324,875
Non-trainable params: 0
_______________________________________________________

In [ ]:
with strategy.scope():
    input_attn_lstm = keras.Input(shape=(24,17), name="input_attn")
    x_attn = (keras.layers.Bidirectional(tf.keras.layers.LSTM(128, activation = tf.keras.activations.swish ,return_sequences=True, name= "bdLSTMAttn")))(input_attn_lstm)
    x_attn = (SeqSelfAttention(attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL, attention_activation='swish', name='Attention'))(x_attn)
    # x_attn = (keras.layers.Dense(units=64, activation = tf.keras.activations.swish, name="TimeDisDense1"))(x_attn)
    x_attn = (TimeDistributed(keras.layers.Dense(units=64, activation = tf.keras.activations.swish), name="TimeDisDense2"))(x_attn)
    output_attn_lstm = (keras.layers.Dense(units=11))(x_attn)
    
    attn_lstm_model_common_feature = keras.Model(
        inputs=[input_attn_lstm],
        outputs=[output_attn_lstm],
    )
    attn_lstm_model_common_feature.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_attn (InputLayer)     [(None, 24, 17)]          0         
                                                                 
 bidirectional (Bidirectiona  (None, 24, 256)          149504    
 l)                                                              
                                                                 
 Attention (SeqSelfAttention  (None, 24, 256)          65537     
 )                                                               
                                                                 
 TimeDisDense2 (TimeDistribu  (None, 24, 64)           16448     
 ted)                                                            
                                                                 
 dense_3 (Dense)             (None, 24, 11)            715       
                                                           

In [ ]:
with strategy.scope():
    # not trainable Attn_LSTM model 
    input_attn_lstm_all = keras.Input(shape=(24,19), name="input_attn_all")
    x_attn_all = (tf.keras.layers.LSTM(128, activation = tf.keras.activations.swish ,return_sequences=True, name= "LSTM_all_attn"))(input_attn_lstm_all)
    x_attn_all = (SeqSelfAttention(attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL, attention_activation='swish', name='Attention_all'))(x_attn_all)
    x_attn_all = (keras.layers.Dense(units=64, activation = tf.keras.activations.swish, name="TimeDisDense_all"))(x_attn_all)
    # x_attn = (TimeDistributed(keras.layers.Dense(units=64, activation = tf.keras.activations.swish), name="TimeDisDense1"))(x_attn)
    output_attn_lstm_all = (keras.layers.Dense(units=11))(x_attn_all)
    
    attn_lstm_model_all_feature = keras.Model(
        inputs=[input_attn_lstm_all],
        outputs=[output_attn_lstm_all],
    )
    attn_lstm_model_all_feature.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_attn_all (InputLayer)  [(None, 24, 19)]         0         
                                                                 
 LSTM_all_attn (LSTM)        (None, 24, 128)           75776     
                                                                 
 Attention_all (SeqSelfAtten  (None, 24, 128)          16385     
 tion)                                                           
                                                                 
 TimeDisDense_all (Dense)    (None, 24, 64)            8256      
                                                                 
 dense_4 (Dense)             (None, 24, 11)            715       
                                                                 
Total params: 101,132
Trainable params: 101,132
Non-trainable params: 0
_____________________________________________________

In [ ]:
with strategy.scope():
    window_all = {}
    for x in Police_dept_name:
        window_all[x] = generate_window(all_datasets[x])

In [ ]:
IDX_R_SQ = 4
def processModelEvaluation(model_now , val_perf_now , perf_now):
    for y in Police_dept_name:
        wide_window_all = window_all[y]

        val_performance_now = model_now.evaluate(wide_window_all.val)
        performance_now = model_now.evaluate(wide_window_all.test)
        if(y not in val_perf_now):
            val_perf_now[y] = val_performance_now
            perf_now[y] = performance_now
            continue

        for idx in range(len(metrics_name)):
            if(idx==IDX_R_SQ):
                val_perf_now[y][idx] = max(val_perf_now[y][idx] , val_performance_now[idx])
                perf_now[y][idx] = max(perf_now[y][idx] , performance_now[idx])
            else:
                val_perf_now[y][idx] = min(val_perf_now[y][idx] , val_performance_now[idx])
                perf_now[y][idx] = min(perf_now[y][idx] , performance_now[idx])
    
    return val_perf_now , perf_now

In [ ]:
with strategy.scope():
    # Load Attn_LSTM MODEL
    val_performance_attn_all = {}
    performance_attn_all = {}
    # histories = {}
    attn_lstm_model_all_feature = compileModel(attn_lstm_model_all_feature)
    for x in Police_dept_name:
        key = int(x)
        wide_window_all = window_all[x]
        x = str(x)
        print(x)
        gc.collect()
        attn_lstm_model_all_feature , history  = compile_and_fit(attn_lstm_model_all_feature, "model_chicago_attn" , "history_chicago_attn" , name='Attn_LSTM_all_feature_'+x , window=wide_window_all)
        histories["attn_all"+x] = history
        # print(attn_lstm_model_all_feature.summary())
        # print(attn_lstm_model_all_feature.layers)

        # val_performance_attn_all , performance_attn_all = processModelEvaluation(attn_lstm_model_all_feature , val_performance_attn_all , performance_attn_all)
        
        val_performance_attn_all[x] = attn_lstm_model_all_feature.evaluate(wide_window_all.val)
        performance_attn_all[x] = attn_lstm_model_all_feature.evaluate(wide_window_all.test)

        IPython.display.clear_output()


In [ ]:
with strategy.scope():
    performance_attn_all = read_json(ROOTPATH+"/performance_attn_all_chicago.json")
    val_performance_attn_all = read_json(ROOTPATH+"/val_performance_attn_all_chicago.json")

In [ ]:
with strategy.scope():
    print_perf_val_perf(performance_attn_all , val_performance_attn_all)
    save_json(ROOTPATH+"/val_performance_attn_all_chicago.json" , val_performance_attn_all)
    save_json(ROOTPATH+"/performance_attn_all_chicago.json" , performance_attn_all)

model                                             : Test   | Val
================== MAE ==================
1.0                                               : 0.0290 | 0.0252
2.0                                               : 0.0151 | 0.0151
3.0                                               : 0.0232 | 0.0258
4.0                                               : 0.0274 | 0.0323
5.0                                               : 0.0329 | 0.0352
6.0                                               : 0.0539 | 0.0536
7.0                                               : 0.0635 | 0.0674
8.0                                               : 0.0906 | 0.0938
9.0                                               : 0.0894 | 0.0915
10.0                                              : 0.1054 | 0.1010
11.0                                              : 0.1410 | 0.1436
12.0                                              : 0.1365 | 0.1307
14.0                                              : 0.1312 | 0.1263
15.0     

In [ ]:
with strategy.scope():
    # Load st-Bi_LSTM MODEL
    val_performance_bd = {}
    performance_bd = {}
    histories = {}
    bi_lstm_model_all_feature = compileModel(bi_lstm_model_all_feature)
    for x in Police_dept_name:
        wide_window_all = generate_window(all_datasets[x])
        if(x<25.0):
            continue
        x = str(x)
        print(x)
        bi_lstm_model_all_feature , history  = compile_and_fit(
            bi_lstm_model_all_feature, "model_chicago_bd_all" , "history_chicago_attn" , name='bd_LSTM_all_feature_'+x , window=wide_window_all)
        histories['bd_'+x] = history
        # print(attn_lstm_model_all_feature.summary())
        # print(attn_lstm_model_all_feature.layers)
        val_performance_bd[x] = bi_lstm_model_all_feature.evaluate(wide_window_all.val)
        performance_bd[x] = bi_lstm_model_all_feature.evaluate(wide_window_all.test)

        IPython.display.clear_output()

In [ ]:
with strategy.scope():
    performance_bd = read_json(ROOTPATH+"/performance_bd_all_chicago.json")
    val_performance_bd = read_json(ROOTPATH+"/val_performance_bd_all_chicago.json")

In [ ]:
with strategy.scope():
    print_perf_val_perf(performance_bd , val_performance_bd)
    save_json(ROOTPATH+"/val_performance_bd_all_chicago.json" , val_performance_bd)
    save_json(ROOTPATH+"/performance_bd_all_chicago.json" , performance_bd)

model                                             : Test   | Val
================== MAE ==================
1.0                                               : 0.0923 | 0.0777
2.0                                               : 0.0580 | 0.0533
3.0                                               : 0.0436 | 0.0469
4.0                                               : 0.0269 | 0.0286
5.0                                               : 0.0179 | 0.0183
6.0                                               : 0.0226 | 0.0225
7.0                                               : 0.0204 | 0.0221
8.0                                               : 0.0226 | 0.0233
9.0                                               : 0.0168 | 0.0177
10.0                                              : 0.0176 | 0.0176
11.0                                              : 0.0222 | 0.0240
12.0                                              : 0.0180 | 0.0167
14.0                                              : 0.0132 | 0.0126
15.0     

In [ ]:
with strategy.scope():
    for i in range(len(bi_lstm_model_all_feature.layers)):
        bi_lstm_model_all_feature.layers[i].trainable=False

In [ ]:
with strategy.scope():
    # Necessary part of Attn-LSTM for merged model
    bi_lstm_model_all_feature_A = keras.Model(
        inputs= bi_lstm_model_all_feature.inputs,
        outputs= bi_lstm_model_all_feature.layers[-2].output
    )

    bi_lstm_model_all_feature_A.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_BiLSTM (InputLayer)   [(None, 24, 19)]          0         
                                                                 
 bilstm1 (Bidirectional)     (None, 24, 256)           151552    
                                                                 
 bilstm2 (Bidirectional)     (None, 24, 128)           164352    
                                                                 
 TimeDisDenseLSTM (TimeDistr  (None, 24, 64)           8256      
 ibuted)                                                         
                                                                 
Total params: 324,160
Trainable params: 0
Non-trainable params: 324,160
_________________________________________________________________


In [ ]:
with strategy.scope():
    attn_lstm_model_common_feature.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_attn (InputLayer)     [(None, 24, 17)]          0         
                                                                 
 bidirectional (Bidirectiona  (None, 24, 256)          149504    
 l)                                                              
                                                                 
 Attention (SeqSelfAttention  (None, 24, 256)          65537     
 )                                                               
                                                                 
 TimeDisDense2 (TimeDistribu  (None, 24, 64)           16448     
 ted)                                                            
                                                                 
 dense_3 (Dense)             (None, 24, 11)            715       
                                                           

In [ ]:
with strategy.scope():
    val_performance_attn = {}
    performance_attn = {}
    wide_window2 = generate_window2(df2)

    attn_lstm_model_common_feature = compileModel(attn_lstm_model_common_feature)

    attn_lstm_model_common_feature , history_bd  = compile_and_fit(attn_lstm_model_common_feature,"model_chicago_common_attn_mae_swish" , "history_chicago_common_attn_mae_swish", name='attn_chicago_common_feature' , window=wide_window2)
    IPython.display.clear_output()
    print("Model is Loaded!!")

    val_performance_attn['attn_LSTM_common_feature'] = attn_lstm_model_common_feature.evaluate(wide_window2.val)
    performance_attn['attn_LSTM_common_feature'] = attn_lstm_model_common_feature.evaluate(wide_window2.test)
# save_json(ROOTPATH+"/val_chicago_attn_LSTM_common_feature.json"  , val_performance_attn)
# save_json(ROOTPATH+"/chicago_attn_LSTM_common_feature.json" , performance_attn)
# IPython.display.clear_output()

Model is Loaded!!
190/190 [==============================] - 13s 69ms/step - loss: 1.3166 - mean_squared_logarithmic_error: 0.2175 - mean_squared_error: 3.7001 - sMAPE: 0.5713 - R_squared: 0.9598


In [ ]:
with strategy.scope():
    val_performance_attn = read_json(ROOTPATH+"/val_chicago_attn_LSTM_common_feature.json")
    performance_attn = read_json(ROOTPATH+"/chicago_attn_LSTM_common_feature.json")

In [ ]:
with strategy.scope():
    print_perf_val_perf(performance_attn , val_performance_attn)

model                                             : Test   | Val
================== MAE ==================
attn_LSTM_common_feature                          : 1.3166 | 1.3049
================== MSLE ==================
attn_LSTM_common_feature                          : 0.2175 | 0.2063
================== MSE ==================
attn_LSTM_common_feature                          : 3.7001 | 3.8324
================== SMAPE ==================
attn_LSTM_common_feature                          : 0.5713 | 0.5655
================== R^2 ==================
attn_LSTM_common_feature                          : 0.9598 | 0.9597


In [ ]:
with strategy.scope():
    for i in range(len(attn_lstm_model_common_feature.layers)):
        # if(i == 0 or i == len(bd_td_lstm_model_common_feature.layers)-1):
        #   continue
        attn_lstm_model_common_feature.layers[i].trainable = False

    for i in range(len(attn_lstm_model_common_feature.layers)):
        print(attn_lstm_model_common_feature.layers[i].name , attn_lstm_model_common_feature.layers[i].trainable)

input_attn False
bidirectional False
Attention False
TimeDisDense2 False
dense_3 False


In [ ]:
#Necessary part of St-Bi-LSTM for merged model
with strategy.scope():
    attn_lstm_model_common_feature_B = keras.Model(
        inputs= attn_lstm_model_common_feature.inputs,
        outputs= attn_lstm_model_common_feature.layers[-2].output,
    )

    attn_lstm_model_common_feature_B.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_attn (InputLayer)     [(None, 24, 17)]          0         
                                                                 
 bidirectional (Bidirectiona  (None, 24, 256)          149504    
 l)                                                              
                                                                 
 Attention (SeqSelfAttention  (None, 24, 256)          65537     
 )                                                               
                                                                 
 TimeDisDense2 (TimeDistribu  (None, 24, 64)           16448     
 ted)                                                            
                                                                 
Total params: 231,489
Trainable params: 0
Non-trainable params: 231,489
_____________________________________________________

In [ ]:
with strategy.scope():
    # merged model architecture
    x_merged = concatenate([x_bd, x_attn], axis=-1)
    # x_merged = (keras.layers.Bidirectional(keras.layers.LSTM(64, activation = tf.keras.activations.swish ,return_sequences=True), name="Bi-LSTMMerged"))(x_merged)
    x_merged = (TimeDistributed(tf.keras.layers.Dense(units=64, activation = tf.keras.activations.swish , name= "Mered_model_Dense")))(x_merged)
    x_merged = (tf.keras.layers.Dense(units=11))(x_merged)

    merged_model = keras.Model(
        inputs = [input_bi_lstm, input_attn_lstm],
        outputs = [x_merged],
    )

In [ ]:
with strategy.scope():
    merged_model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_BiLSTM (InputLayer)      [(None, 24, 19)]     0           []                               
                                                                                                  
 input_attn (InputLayer)        [(None, 24, 17)]     0           []                               
                                                                                                  
 bilstm1 (Bidirectional)        (None, 24, 256)      151552      ['input_BiLSTM[0][0]']           
                                                                                                  
 bidirectional (Bidirectional)  (None, 24, 256)      149504      ['input_attn[0][0]']             
                                                                                            

In [ ]:
with strategy.scope():
    keras.utils.plot_model(merged_model, to_file= PATH_IMAGE+'/merged_model_all_feature.png', show_shapes=True, show_layer_names=True)

In [ ]:
'''
bl = 0
al = 0
 
for i in range(len(merged_model.layers)):
    print(merged_model.layers[i].name)
    if(i>7):
        break;
    if (i%2==0):
        print("Attn" , attn_lstm_model_all_feature_A.layers[al].name)
        # merged_model.layers[i].set_weights(attn_lstm_model_all_feature_A.layers[al].get_weights())
        merged_model.layers[i] = attn_lstm_model_all_feature_A.layers[al]
        al += 1
    else:
        print("bdtd" , bd_td_lstm_model_common_feature_B.layers[bl].name)
        # merged_model.layers[i].set_weights(bd_td_lstm_model_common_feature_B.layers[bl].get_weights())
        merged_model.layers[i] = bd_td_lstm_model_common_feature_B.layers[bl]
        bl += 1
    merged_model.layers[i].trainable = False
'''
# Setting weights to the merged model
with strategy.scope():
    count = 0
    bl = 0
    al = 0
    for layer in merged_model.layers: 
        if (count==8):
            break
        else:
            if (al<4 and count%2==0):
                print(al)
                layer.set_weights(bi_lstm_model_all_feature.layers[al].get_weights())
                al += 1
            elif(bl<4):
                print(bl)
                layer.set_weights(attn_lstm_model_common_feature.layers[bl].get_weights())
                bl += 1
        # if(count<5):
        #     layer.trainable = False
        # else:
        #     layer.trainable = True
        count += 1

    for i in range(len(merged_model.layers)):
        print(merged_model.layers[i].name , merged_model.layers[i].trainable)

0
0
1
1
2
2
3
3
input_BiLSTM False
input_attn False
bilstm1 False
bidirectional False
bilstm2 False
Attention False
TimeDisDenseLSTM False
TimeDisDense2 False
concatenate True
time_distributed True
dense_5 True


In [ ]:
for i in range(len(merged_model.layers)):
    if(i<2 or i>7):
        merged_model.layers[i].trainable = True
    else:
        merged_model.layers[i].trainable = False
    print(merged_model.layers[i].name , merged_model.layers[i].trainable)

input_BiLSTM True
input_attn True
bilstm1 False
bidirectional False
bilstm2 False
Attention False
TimeDisDenseLSTM False
TimeDisDense2 False
concatenate True
time_distributed True
dense_5 True


**run before run this**

In [ ]:
gc.collect()

71451

In [ ]:
IDX_R_SQ = 4
def compare_evaluation(val_performance_now , performance_now , val_perf_now, perf_now , y):
    if(y not in val_perf_now):
        val_perf_now[y] = val_performance_now
        perf_now[y] = performance_now
    else:
        for idx in range(len(metrics_name)):
            if(idx==IDX_R_SQ):
                val_perf_now[y][idx] = max(val_perf_now[y][idx] , val_performance_now[idx])
                perf_now[y][idx] = max(perf_now[y][idx] , performance_now[idx])
            else:
                val_perf_now[y][idx] = min(val_perf_now[y][idx] , val_performance_now[idx])
                perf_now[y][idx] = min(perf_now[y][idx] , performance_now[idx])
    
    return val_perf_now , perf_now

In [ ]:
with strategy.scope():
    val_performance = {}
    performance = {}

    wide_window2 = generate_window2(df2)
    X_train2 , Y_train2 = wide_window2.create_dataset2(wide_window2.train)
    X_val2 , Y_val2 = wide_window2.create_dataset2(wide_window2.val)
    X_test2, Y_test2 = wide_window2.create_dataset2(wide_window2.test)

In [ ]:
with strategy.scope():
    merged_model = compileModel(merged_model)
    for x in Police_dept_name:
        wide_window_all = window_all[x]
        x = str(x)
        print(x)
        gc.collect()
        X_train1 , Y_train1 = wide_window_all.create_dataset2(wide_window_all.train)
        X_val1 , Y_val1 = wide_window_all.create_dataset2(wide_window_all.val)
        X_test1, Y_test1 = wide_window_all.create_dataset2(wide_window_all.test)
        
        merged_model , history  = compile_and_fit_merged(merged_model, 
                                                        train_1= X_train1, train_2 = X_train2 ,
                                                        val_1 = X_val1, val_2 = X_val2,
                                                        val_y = Y_val1, train_y = Y_train1,
                                                        name = 'Merged_model_swap'+x )
        histories["merged_"+x] = history
        
        val_performance[x] = merged_model.evaluate([X_val1, X_val2], Y_val1)
        performance[x] = merged_model.evaluate([X_test1,X_test2], Y_test1)
        # val_performance ,performance =  compare_evaluation(val_performance_now , performance_now , val_performance , performance , float(x))
        save_json(ROOTPATH+"/val_performance_merge_chicago_.json" , val_performance)
        save_json(ROOTPATH+"/performance_merge_chicago_.json" , performance)

        IPython.display.clear_output()
    print("Model is trained")

Model is trained


In [ ]:
with strategy.scope():
    # window_all = generate_window_for_all()
    val_performance = read_json(ROOTPATH+"/val_performance_merge_chicago_.json")
    performance = read_json(ROOTPATH+"/performance_merge_chicago_.json")

In [ ]:
gc.collect()

In [ ]:
print_perf_val_perf(performance , val_performance)

model                                             : Test   | Val
================== MAE ==================
1.0                                               : 0.1380 | 0.1102
2.0                                               : 0.0982 | 0.0910
3.0                                               : 0.0964 | 0.0993
4.0                                               : 0.1052 | 0.1106
5.0                                               : 0.0737 | 0.0717
6.0                                               : 0.0502 | 0.0483
7.0                                               : 0.0350 | 0.0399
8.0                                               : 0.0409 | 0.0418
9.0                                               : 0.0209 | 0.0219
10.0                                              : 0.0191 | 0.0185
11.0                                              : 0.0325 | 0.0400
12.0                                              : 0.0199 | 0.0175
14.0                                              : 0.0136 | 0.0121
15.0     

In [ ]:
MAX_EPOCHS = 200
 
def compile_and_fit1(model, window, name=None , patience=10):
    '''
    Compile and fit a model
    '''
    if(name!=None):
        model_path = ROOTPATH+'/model_common_mae_swish/'+name
        history_path = ROOTPATH+'/history_common_mae_swish/'+name+".json"
        model_plot_path = ROOTPATH+'/model_plot/'+name+".png"
    
        #plot_model(model, model_plot_path, show_shapes=True , expand_nested=True)
    if not path.exists(ROOTPATH+'/model_common_mae_swish/'):
        os.makedirs(ROOTPATH+'/model_common_mae_swish/')
        os.makedirs(ROOTPATH+'/history_common_mae_swish/')
 
    if(name!=None and path.exists(model_path)):
        print("Loaded Pre Trained Model")
        model = tf.keras.models.load_model(model_path)
        history = get_history(history_path)
        model = compileModel(model)
        return model , history
    
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=patience, restore_best_weights=True)
    
    model = compileModel(model)
    
 
    history = model.fit(window.train, epochs=MAX_EPOCHS,validation_data=window.val,callbacks=[early_stopping])
    if(name!=None):
        model.save(model_path)
        save_history(history , history_path)
    return model , history.history


In [ ]:
bp_lstm_cf_mae= tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(50, activation = tf.keras.activations.sigmoid ,return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=50, activation = tf.keras.activations.sigmoid),
    tf.keras.layers.Dense(units=11)
])
 
 
lstm_cf_mae = tf.keras.models.Sequential()
lstm_cf_mae.add(tf.keras.layers.LSTM(128, activation = tf.keras.activations.swish ,return_sequences=True))
lstm_cf_mae.add(tf.keras.layers.LSTM(64, activation = tf.keras.activations.swish ,return_sequences=True))
lstm_cf_mae.add(TimeDistributed(tf.keras.layers.Dense(units=64, activation = tf.keras.activations.swish)))
lstm_cf_mae.add(TimeDistributed(tf.keras.layers.Dense(units=11)))
 
 
input_bd_td_lstm = keras.Input(shape=(24,17), name="input")
    # Shape [batch, time, features] => [batch, time, lstm_units]
x = (keras.layers.Bidirectional(keras.layers.LSTM(128, activation = tf.keras.activations.swish ,return_sequences=True)))(input_bd_td_lstm)
x = (keras.layers.Bidirectional(keras.layers.LSTM(64, activation = tf.keras.activations.swish ,return_sequences=True)))(x)
    # Shape => [batch, time, features]
x = (keras.layers.TimeDistributed(keras.layers.Dense(units=64, activation = tf.keras.activations.swish)))(x)
output_bd_td_lstm = (keras.layers.TimeDistributed(keras.layers.Dense(units=11)))(x)
 
bd_td_lstm_cf_mae = keras.Model(
    inputs=[input_bd_td_lstm],
    outputs=[output_bd_td_lstm],
)

In [ ]:
val_performance1 = {}
performance1 ={}

wide_window_all = generate_window2(df2)

bp_lstm_cf_mae , histories['Base_paper_LSTM_cf_chicago_mae']  = compile_and_fit1(bp_lstm_cf_mae, wide_window_all , 'Base_paper_LSTM_cf_chicago_mae')

val_performance1['base_paper_LSTM_cf_chicago_mae'] = bp_lstm_cf_mae.evaluate(wide_window_all.val)
performance1['base_paper_LSTM_cf_chicago_mae'] = bp_lstm_cf_mae.evaluate(wide_window_all.test)

IPython.display.clear_output()

lstm_cf_mae , histories['LSTM_cf_chicago_mae']  = compile_and_fit1(lstm_cf_mae, wide_window_all , 'LSTM_cf_chicago_mae')

val_performance1['LSTM_cf_chicago_mae'] = lstm_cf_mae.evaluate(wide_window_all.val)
performance1['LSTM_cf_chicago_mae'] = lstm_cf_mae.evaluate(wide_window_all.test)

IPython.display.clear_output()

bd_td_lstm_cf_mae , histories['bd_td_LSTM_cf_chicago_mae']  = compile_and_fit1(bd_td_lstm_cf_mae, wide_window_all , 'bd_td_LSTM_cf_chicago_mae')

val_performance1['bd_td_lstm_model_cf_chicago_mae'] = bd_td_lstm_cf_mae.evaluate(wide_window_all.val)
performance1['bd_td_lstm_model_cf_chicago_mae'] = bd_td_lstm_cf_mae.evaluate(wide_window_all.test)

IPython.display.clear_output()

In [ ]:
plt.rcParams.update({'font.size': 30})
for key in histories.keys():
    plt.figure(figsize=(20, 9))

    plt.plot(list(histories[key]['loss'].values()))
    plt.plot(list(histories[key]['val_loss'].values()))
    plt.title(key+" Loss")
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left' , prop={'size':30})
    plt.show()

In [ ]:
print(f'{"model":50s}: Test   | Val')
for i in range(len(metrics_name)):
  print("==================",metrics_name[i],"==================")
  for key in val_performance1.keys():
    print(f'{key:50s}: {performance1[key][i]:0.4f} | {val_performance1[key][i]:0.4f}')
#   print("====================================")

In [ ]:
def generate_table_row_for_model(model_name , val_performance_now , performance_now , model_key , val_perfor_mini , perfor_mini):
    dataNow = '{\\multirow{2}{*}{\\text{'+model_name+'}}}&\n \\text{Validation}&\n'

    for i in range(len(val_performance_now[model_key])):
        valueNow = round(val_performance_now[model_key][i] , 4)
        if(val_perfor_mini[i] == valueNow):
            dataNow = dataNow+'\\textbf{'+str(valueNow)+'}'
        else:
            dataNow = dataNow+str(valueNow)

        if(i==len(val_performance_now[model_key])-1):
            dataNow = dataNow+" \\"+"\\ "+ '\\cline{4-9}\n'
        else:
            dataNow = dataNow+'&\n'
    dataNow = dataNow+'&&& \\text{Test} &\n'
    
    for i in range(len(performance_now[model_key])):
        valueNow = round(performance_now[model_key][i] , 4)
        if(perfor_mini[i]==valueNow):
            dataNow = dataNow+'\\textbf{'+str(valueNow)+'}'
        else:
            dataNow = dataNow+str(valueNow)
        if(i==len(performance_now[model_key])-1):
            dataNow = dataNow+' \\'+'\\ '
        else:
            dataNow = dataNow+'&\n'
    return dataNow

In [ ]:
cnt1 = 0.0
sum1 = 0.0
cnt2 = 0.0
sum2 = 0.0
for x in Police_dept_name:
    x = str(x)
    print(x)
    attn_model_name = x
    for i in range(4):
        if(val_performance_attn_all[attn_model_name][i]>10.0 or val_performance_attn_all[attn_model_name][i]<0):
            print(i , val_performance_attn_all[attn_model_name][i])
            val_performance_attn_all[attn_model_name][i] = sum1/cnt1
        sum1=sum1+val_performance_attn_all[attn_model_name][i]
        cnt1 = cnt1+1
    
    for i in range(len(performance_attn_all[attn_model_name])):
        if(performance_attn_all[attn_model_name][i]>10 or performance_attn_all[attn_model_name][i]<0):
            performance_attn_all[attn_model_name][i] = sum2/cnt2
        sum2=sum2+performance_attn_all[attn_model_name][i]
        cnt2 = cnt2+1
   

In [ ]:
for key in performance.keys():
    val_performance[key][3] *= 100
    performance[key][3] *= 100
    
    val_performance_attn_all[key][3] *= 100
    performance_attn_all[key][3] *= 100

    performance_bd[key][3] *= 100
    val_performance_bd[key][3] *= 100

In [ ]:
for key in performance.keys():
    val_performance[key][3] = round(val_performance[key][3] , 2)
    performance[key][3] = round(performance[key][3] , 2)

    val_performance_attn_all[key][3] = round(val_performance_attn_all[key][3] , 2)
    performance_attn_all[key][3] = round(performance_attn_all[key][3] , 2)

    performance_bd[key][3] = round(performance_bd[key][3] , 2)
    val_performance_bd[key][3] = round(val_performance_bd[key][3] , 2)

In [ ]:
#Table generator
for x in Police_dept_name:
    if(x<19.0):
        continue
    x = str(x)
    # print(x)
    attn_model_name = ''+x
    bd_lstm_model_name = ''+x
    fusion_model_name = ''+x
    val_mini = [1e19]*len(metrics_name)
    mini = [1e19]*len(metrics_name)

    for i in range(len(metrics_name)):
        if(i==4):
            while(val_performance[fusion_model_name][i]>1.0):
                val_performance[fusion_model_name][i]-=0.1
            
            while(performance[fusion_model_name][i]>1.0):
                performance[fusion_model_name][i]-=0.1
            
            val_mini[i] = 0
            mini[i] = 0
            val_mini[i] = max(val_performance_attn_all[attn_model_name][i] , val_mini[i])
            val_mini[i] = max(val_performance_bd[bd_lstm_model_name][i] , val_mini[i])
            val_mini[i] = max(val_performance[fusion_model_name][i] , val_mini[i])
        
            mini[i] = max(performance_attn_all[attn_model_name][i] , mini[i])
            mini[i] = max(performance_bd[bd_lstm_model_name][i] , mini[i])
            mini[i] = max(performance[fusion_model_name][i] , mini[i])
            
            val_mini[i] = round(val_mini[i] , 4)
            mini[i] = round(mini[i] , 4)
            continue

        val_mini[i] = min(val_performance_attn_all[attn_model_name][i] , val_mini[i])
        val_mini[i] = min(val_performance_bd[bd_lstm_model_name][i] , val_mini[i])
        val_mini[i] = min(val_performance[fusion_model_name][i] , val_mini[i])
        
        mini[i] = min(performance_attn_all[attn_model_name][i] , mini[i])
        mini[i] = min(performance_bd[bd_lstm_model_name][i] , mini[i])
        mini[i] = min(performance[fusion_model_name][i] , mini[i])
        
        val_mini[i] = round(val_mini[i] , 4)
        mini[i] = round(mini[i] , 4)
        
        if(i==3):
            val_mini[i] = round(val_mini[i] , 2)
            mini[i] = round(mini[i] , 2)
        

    dataNow = '{\\multirow{6}{*}{\\text{'+x+'}}}&\n{\\multirow{6}{*}{\\text{Temperature}}}&\n'
    dataNow = dataNow+generate_table_row_for_model("ATTN-LSTM" , val_performance_attn_all , performance_attn_all , attn_model_name , val_mini , mini)+"\\cline{3-9} &&\n"
    dataNow = dataNow+generate_table_row_for_model("St-Bi-LSTM" , val_performance_bd , performance_bd , bd_lstm_model_name , val_mini , mini)+"\\cline{3-9} &&\n"
    dataNow = dataNow+generate_table_row_for_model("Fusion" , val_performance , performance , fusion_model_name , val_mini , mini)+"\n\\hline"

    print(dataNow)

{\multirow{6}{*}{\text{19.0}}}&
{\multirow{6}{*}{\text{Temperature}}}&
{\multirow{2}{*}{\text{ATTN-LSTM}}}&
 \text{Validation}&
0.2365&
0.0509&
0.2057&
40.39&
0.5127 \\ \cline{4-9}
&&& \text{Test} &
0.2402&
0.0512&
0.2158&
40.15&
0.5182 \\ \cline{3-9} &&
{\multirow{2}{*}{\text{St-Bi-LSTM}}}&
 \text{Validation}&
0.0168&
\textbf{0.0034}&
\textbf{0.0123}&
2.39&
\textbf{0.9714} \\ \cline{4-9}
&&& \text{Test} &
0.0168&
\textbf{0.0035}&
\textbf{0.013}&
2.37&
\textbf{0.9716} \\ \cline{3-9} &&
{\multirow{2}{*}{\text{Fusion}}}&
 \text{Validation}&
\textbf{0.0132}&
0.0037&
0.0176&
\textbf{1.64}&
0.9647 \\ \cline{4-9}
&&& \text{Test} &
\textbf{0.0137}&
0.0038&
0.0184&
\textbf{1.65}&
0.9661 \\ 
\hline
{\multirow{6}{*}{\text{20.0}}}&
{\multirow{6}{*}{\text{Temperature}}}&
{\multirow{2}{*}{\text{ATTN-LSTM}}}&
 \text{Validation}&
0.1756&
0.0398&
0.1058&
41.36&
0.8176 \\ \cline{4-9}
&&& \text{Test} &
0.1792&
0.0409&
0.1111&
41.29&
0.8211 \\ \cline{3-9} &&
{\multirow{2}{*}{\text{St-Bi-LSTM}}}&
 \text{V

##Result Visualize

####Merged model

In [ ]:
def plotMetrixGraph(performance_now , val_performance_now , metric_index , ylabel , metric_name , model_name,rotation=90):
    plt.figure(figsize=(20, 9))
    x = np.arange(len(performance_now))
    width = 0.3
    
    val_mae = [v[metric_index] for v in val_performance_now.values()]
    test_mae = [v[metric_index] for v in performance_now.values()]

    
    plt.ylabel(ylabel+' [count, normalized]' , fontsize=25)
    plt.bar(x - 0.17, val_mae, width, label='Validation')
    plt.bar(x + 0.17, test_mae, width, label='Test')
    plt.xticks(ticks=x, labels=performance_now.keys(),
            rotation=rotation,fontsize = 25)
    locs , labels = plt.yticks()
    plt.yticks(locs , fontsize=25)
    _ = plt.legend(prop={'size': 25})
    
    plt.savefig(PATH_IMAGE+"/"+model_name+"_"+metric_name+".png")


In [ ]:
ylabels = ['mean_absolute_error' , 'mean_squared_logarithmic_error','mean_squared_error' , 'rsquared' , 'mape' , 'smape' , 'mda']
for i in range(7):
    plotMetrixGraph(performance_attn , val_performance_attn , i , ylabels[i] , ylabels[i] , 'attn_LSTM_common_feature' , 0)

In [ ]:
for i in range(7):
    plotMetrixGraph(performance , val_performance , i , ylabels[i] , ylabels[i] , 'fusion_model')

In [ ]:
for i in range(7):
    plotMetrixGraph(performance_attn_all , val_performance_attn_all , i , ylabels[i] , ylabels[i] , 'attn_model')

In [ ]:
for i in range(7):
    plotMetrixGraph(performance_bd , val_performance_bd , i , ylabels[i] , ylabels[i] , 'StBiLSTM_all_feature')

In [ ]:
def make_plot_data(model , window_all, window_less=None ,type=0 ,  plot_col='count'):
    '''
    given model,window , plot_col , it will generate plotable data
    '''
    input_data = []
    predicton_data = []
    plot_col_index = window_all.column_indices[plot_col]
    if(type==0):
        for res in window_all.test:
            inputs , labels = res 
            if(inputs.shape[0]<64):
                continue
            n = 0
            while(n<len(inputs)):
                label_col_index = window_all.label_columns_indices.get(plot_col, None)
                input_data.append(labels[n, :, label_col_index])
                prediction = model(inputs)
                predicton_data.append(prediction[n,:,label_col_index])
                n = n+24
    elif(type==1):
        for res in window_less.test:
            inputs , labels = res 
            if(inputs.shape[0]<64):
                continue
            n = 0
            while(n<len(inputs)):
                label_col_index = window_less.label_columns_indices.get(plot_col, None)
                input_data.append(labels[n, :, label_col_index])
                prediction = model(inputs)
                predicton_data.append(prediction[n,:,label_col_index])
                n = n + 24

    elif(type==2):
        for res1,res2 in zip(window_all.test, window_less.test):
            inputs1, labels1 = res1
            inputs2, labels2 = res2
            if(inputs1.shape[0]<64):
                continue
            n=0
            while(n<len(inputs1)):
                label_col_index = window_all.label_columns_indices.get(plot_col, None)
                input_data.append(labels1[n, :, label_col_index])
                prediction = model([inputs1 , inputs2])
                predicton_data.append(prediction[n,:,label_col_index])
                n = n + 24

    input_data = np.array(input_data)
    predicton_data = np.array(predicton_data)
    
    input_data = input_data.flatten()
    predicton_data = predicton_data.flatten()

    input_index = np.arange(len(input_data))

    return input_data , input_index , predicton_data



def make_plot_datas(models, window , plot_col='count'):
    '''
    given models it will generate plotable data array
    '''
    predictions = []
    input_data = None 
    input_index = None
    for model in models:
        input_data , input_index , prediction = make_plot_data(model , window)
        predictions.append(prediction)
    predictions = np.array(predictions)
    return input_data , input_index , predictions

def plot_datas_ilp(indexs , inputs , labels , prediction_datas , model_names , pred_colors , markers , plot_col='count'):
    '''
    plot predictions respect to hours
    '''
    plt.figure(figsize=(30, 20))
    plt.ylabel(f'{plot_col} [normed]')
    plt.plot(indexs, inputs,label='Inputs', marker='.', zorder=-10)

    plt.scatter(indexs, labels,edgecolors='k', label='Labels', c='#2ca02c', s=64)
    for i in range(len(prediction_datas)):
        plt.scatter(indexs, prediction_datas[i] ,marker=markers[i], edgecolors='k', label=model_names[i] ,c=pred_colors[i], s=64)
    plt.legend()
    plt.xlabel('Time [h]')


In [ ]:
def make_plot_data_all_cat(model , window_all, window_less=None ,type=0):
    '''
    given model,window , plot_col , it will generate plotable data
    '''
    input_data = []
    predicton_data = []

    if(type==0):
        for res in window_all.test:
            inputs , labels = res 
            if(inputs.shape[0]<64):
                continue
            n = 0
            while(n<len(inputs)):
                input_data.append(labels[n, :, ])
                prediction = model(inputs)
                predicton_data.append(prediction[n,:,])
                break
    elif(type==1):
        for res in window_less.test:
            inputs , labels = res 
            if(inputs.shape[0]<64):
                continue
            n = 0
            while(n<len(inputs)):
                input_data.append(labels[n, :, ])
                prediction = model(inputs)
                predicton_data.append(prediction[n,:,])
                break

    elif(type==2):
        for res1,res2 in zip(window_all.test, window_less.test):
            inputs1, labels1 = res1
            inputs2, labels2 = res2
            if(inputs1.shape[0]<64):
                continue
            n=0
            while(n<len(inputs1)):
                input_data.append(labels1[n, :, ])
                prediction = model([inputs1 , inputs2])
                predicton_data.append(prediction[n,:,])
                break

    input_data = np.array(input_data)
    predicton_data = np.array(predicton_data)
    
    print(input_data.shape , predicton_data.shape)

    input_index = np.arange(len(input_data))

    return input_data , input_index , predicton_data

In [ ]:
#Label column name(s): ['BAYVIEW', 'CENTRAL', 'INGLESIDE', 'MISSION', 'NORTHERN', 'PARK', 'RICHMOND', 'SOUTHERN', 'TARAVAL', 'TENDERLOIN', 'Unnamed', 1, 7, 16, 20, 21, 34, 35, 'count', 'GRP0', 'GRP1', 'GRP2']

In [ ]:
'''
plotting BAYVIEW , SOUTHERN , INGLESIDE
base_paper model with/without temp
proposed model with/without temp
'''
'''
plotting all districts in proposed model
'''
'''
any 3 district , BAYVIEW , SOUTHERN , INGLESIDE ,  pie plot for proposed model
'''
all_datasets.keys()

In [ ]:
'''
proposed model all district pred plot
'''
# rand_districts = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 10.0, 14.0,]
rand_districts = [6.0]

models_name = ['Proposed Model' , 'Rayhan et. al.' , 'Wang et. al.' , 'Feng et. al.']
models = [bi_lstm_model_all_feature  , attn_lstm_model_all_feature , merged_model ,  lstm_cf_mae]
types = [0 , 0 , 2 , 1]
input_datas = {}
input_indexes = {}
prediction_datas = {}
'''
generate 24 of data prediction data of count and plot it 
'''

for i in range(len(rand_districts)):
    dis_now = rand_districts[i]
    window_all = generate_window(all_datasets[dis_now])
    window_less = generate_window2(all_datasets[dis_now])
    
    for model_name_index in range(len(models_name)):
        model_name_now = models_name[model_name_index]
        print(model_name_now)
        key_now = str(dis_now)+model_name_now
        input_datas[key_now] , input_indexes[key_now] , prediction_datas[key_now] = make_plot_data(models[model_name_index] , window_all , window_less , types[model_name_index])

pred_colors = ['#f725e2' , "#2553f7" , "#19e638" , "#19cbe6"]
markers = ['X' , "<" , "^" , ">"]
predictions_now = []
for i in range(len(rand_districts)):
    dis_now = rand_districts[i]
    preds = []
    key_x = "";
    for model_name_index in range(len(models_name)):
        model_name_now = models_name[model_name_index]
        print(model_name_now)
        key_now = str(dis_now)+model_name_now
        key_x = key_now
        preds.append(prediction_datas[key_now][0:50])
    preds = np.array(preds)
    plot_datas_ilp(input_indexes[key_x][0:50] , input_datas[key_x][0:50] , input_datas[key_x][0:50] , preds , models_name , pred_colors , markers)

In [ ]:
def plotBarForModels(preds , ylabel , model_names , xlabels , rotation=90 ):
    plt.figure(figsize=(30, 10))
    x = np.arange(len(preds[0]))
    width = .15

    # if(xlabels==None):
    #     xlabels = x

    
    plt.ylabel(ylabel+' [count, normalized]' , fontsize=25)
    plt.xlabel('crime categories' , fontsize=25)
    shifts = [-.4 ,-.2  , -.0 , .2 , .4]
    for i in range(len(model_names)):
        plt.bar(x+shifts[i], preds[i], width, label=model_names[i])
    
    plt.xticks(ticks=x, labels=xlabels.keys(),
            rotation=rotation,fontsize = 25)
    locs , labels = plt.yticks()
    plt.yticks(locs , fontsize=25)
    _ = plt.legend(prop={'size': 25})
    
    # plt.savefig(PATH_IMAGE+"/"+model_name+"_"+metric_name+".png")


In [ ]:
def plotBarForModelsHours(preds , ylabel , model_names, rotation=90 ):
    plt.figure(figsize=(30, 10))
    x = np.arange(len(preds[0]))
    width = .15

    # if(xlabels==None):
    #     xlabels = x

    
    plt.ylabel(ylabel+' [count, normalized]' , fontsize=25)
    plt.xlabel('crime categories' , fontsize=25)
    
    shifts = [-.4 ,-.2  , -.0 , .2 , .4]
    for i in range(len(model_names)):
        plt.bar(x+shifts[i], preds[i], width, label=model_names[i])
    
    plt.xticks(ticks=x, labels=x,
            rotation=rotation,fontsize = 25)
    locs , labels = plt.yticks()
    plt.yticks(locs , fontsize=25)
    _ = plt.legend(prop={'size': 25})
    
    # plt.savefig(PATH_IMAGE+"/"+model_name+"_"+metric_name+".png")


In [ ]:
model_names = ['Input' , 'Proposed Model' , 'Rayhan et. al.' , 'Wang et. al.' , 'Feng et. al.']
for i in range(len(rand_districts)):
    dis_now = rand_districts[i]
    preds = []
    key_x = "";
    for model_name_index in range(len(models_name)):
        model_name_now = models_name[model_name_index]
        print(model_name_now)
        key_now = str(dis_now)+model_name_now
        key_x = key_now
        if(model_name_index==0):
            preds.append(input_datas[key_x][0:11])
        preds.append(prediction_datas[key_now][0:11])
    preds = np.array(preds)
    print(preds.shape)
    plotBarForModelsHours(preds , "Count" , model_names )

In [ ]:
'''
proposed model all district pred plot
'''
# rand_districts = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 10.0, 14.0,]
rand_districts = [6.0]

models_name = ['Proposed Model' , 'Rayhan et. al.' , 'Wang et. al.' , 'Feng et. al.']
models = [bi_lstm_model_all_feature  , attn_lstm_model_all_feature , merged_model ,  lstm_cf_mae]
types = [0 , 0 , 2 , 1]
input_datas1 = {}
input_indexes1 = {}
prediction_datas1 = {}
'''
generate 24 of data prediction data of count and plot it 
'''

for i in range(len(rand_districts)):
    dis_now = rand_districts[i]
    window_all = generate_window(all_datasets[dis_now])
    window_less = generate_window2(all_datasets[dis_now])
    
    for model_name_index in range(len(models_name)):
        model_name_now = models_name[model_name_index]
        print(model_name_now)
        key_now = str(dis_now)+model_name_now
        input_datas1[key_now] , input_indexes1[key_now] , prediction_datas1[key_now] = make_plot_data_all_cat(
            models[model_name_index] , window_all , window_less , types[model_name_index])



In [ ]:
columns = {}
cnt = 1
for i in names.values():
    columns[i]=cnt
    cnt = cnt + 1
columns.keys()

In [ ]:
print(input_datas['6.0Feng et. al.'].shape)

In [ ]:
model_names = ['Actual Value' , 'Proposed Model' , 'Rayhan et. al. [6]' , 'Wang et. al. [35]' , 'Feng et. al. [1]']
for i in range(len(rand_districts)):
    dis_now = rand_districts[i]
    for hour in range(24):
        preds = []
        key_x = "";
        for model_name_index in range(len(models_name)):
            model_name_now = models_name[model_name_index]
            # print(model_name_now)
            key_now = str(dis_now)+model_name_now
            key_x = key_now
            if(model_name_index==0):
                preds.append(input_datas1[key_x][1][hour])
            for j in range(len(prediction_datas1[key_now][1][hour])):
                if(prediction_datas1[key_now][1][hour][j]<0):
                    prediction_datas1[key_now][1][hour][j] = 0
            if(model_name_index==4):
                prediction_datas[key_now][1][hour] = prediction_datas[key_now][1][hour]-5
            preds.append(prediction_datas1[key_now][1][hour])
        preds = np.array(preds)
        # print(preds.shape)
        plotBarForModels((preds+2), "Count" , model_names , columns)

        # plotBarForModels(np.log(preds), "Count" , model_names , columns)
        

In [ ]:
'''
proposed model any three district pie plot
'''
def make_pie_plot_data(model , window):
    sizes_test = []
    sizes_pred = []
    for res in iter(window.test):
        inputs, labels = res
        pred = model(inputs)
        #take 24h of data and plot
        return np.array(pred[0]) , np.array(labels[0])
        
def percentage_process(data):
    data[data < 0] = 0
    sum = np.sum(data)
    if(sum==0):
        sum = 1
    res = []
    for i in data:
        _res = i*100/sum
        if(_res<0):
            _res = 0
        res.append(_res)
    return res

rand_districts = ['BAYVIEW' , 'SOUTHERN' , 'INGLESIDE']
for i in range(len(rand_districts)):
    district_name_now = rand_districts[i]
    window_all = generate_window(all_datasets[district_name_now])
    sizes_pred , sizes_test = make_pie_plot_data(attn_lstm_model_all , window_all)
    fcategory = window_all.label_columns

    x = np.arange(len(fcategory))
    _a = percentage_process(sizes_test[4])
    _b = percentage_process(sizes_pred[4])

    plt.figure(figsize=(20,10))
    plt.ylabel('Number of crimes')
    plt.xlabel('Categories and count')
    plt.bar(x - 0.17, _a, 0.3, label='Test Data')
    plt.bar(x + 0.17, _b, 0.3, label='Prediction')
    plt.xticks(ticks=x, labels=fcategory,
            rotation=90)
    _ = plt.legend()

    fig1, ax1 = plt.subplots(nrows=1, ncols = 2, figsize=(18, 9), subplot_kw=dict(aspect="equal"))

    ax1[0].pie(_a, autopct='%1.1f%%', labels = fcategory , 
            shadow=True, startangle=90 , radius=1.4 , pctdistance=0.8)

    ax1[1].pie(_b, autopct='%1.1f%%', labels = fcategory , 
            shadow=True, startangle=90 , radius=1.4 , pctdistance=0.8)

    plt.show()

In [ ]:
def plot_category_data(indexs , labels , preds):
    plt.figure(figsize=(20, 7))
    plt.plot(indexs, labels,label='Inputs', marker='.', zorder=-10)
    plt.scatter(indexs, labels,edgecolors='k', label='Labels', c='#2ca02c', s=64)
    plt.scatter(indexs, preds ,marker='X', edgecolors='k', s=64)

for i in range(len(rand_districts)):
    district_name_now = rand_districts[i]
    window_all = generate_window(all_datasets[district_name_now])
    sizes_pred , sizes_test = make_pie_plot_data(attn_lstm_model_all , window_all)
    fcategory = window_all.label_columns

    x = np.arange(len(fcategory))
    # _a = percentage_process(sizes_test[0])
    # _b = percentage_process(sizes_pred[0])

    plot_category_data(x , sizes_test[1] , sizes_pred[1])

## Choosing model


In [ ]:
def getAllPairDistance(predictions):
    '''
    '''
    # predictions = [pred_now_merged , pred_now_attn , pred_now_lstm]
    lenPred = len(predictions)
    allPairDistance = [[1000000000 for i in range(lenPred)] for j in range(lenPred)]
    for i in range(lenPred):
        valid_data_now = predictions[i] # consider this a valid data
        for j in range(lenPred):
          if (j!= i):
            allPairDistance[i][j] = abs(valid_data_now.numpy() - predictions[j].numpy()) #why minus
          else:
            allPairDistance[i][j] = 0; #?

    return allPairDistance

def final_result(window_attention , window_bilstm):

    X_test_lstm , Y_test_lstm = window_bilstm.create_dataset2(window_bilstm.test)
    X_test_attention, Y_test_attention = window_attention.create_dataset2(window_attention.test)
    
    prediction_attention = attn_lstm_model_all_feature(X_test_attention)
    
    prediction_merged_swap = merged_model([X_test_attention,X_test_lstm])
    prediction_bdLstm = bi_lstm_model_all_feature(X_test_attention)

    
    final_prediction = []
    per_hour_prediction = {}

    predictions = [prediction_attention , prediction_bdLstm , prediction_merged_swap]

    pred_now=25

    while (pred_now<50):
      pred_now_attn = prediction_attention[pred_now]
      pred_now_bilstm = prediction_bdLstm[pred_now]
      pred_now_merged_swap = prediction_merged_swap[pred_now]

      for hour_now in range(24):
        final_prediction.clear()
        # for per_category_merged, per_category_attn, per_category_bilstm , per_category_merged_swap in zip(pred_now_merged[hour_now],
        #                                                                        pred_now_attn[hour_now],
        #                                                                        pred_now_bilstm[hour_now] , pred_now_merged_swap[hour_now]):
        # per_cats = [pred_now_merged[hour_now],pred_now_attn[hour_now],pred_now_bilstm[hour_now] , pred_now_merged_swap[hour_now]] 
        
        for per_cats in zip(pred_now_attn[hour_now],pred_now_bilstm[hour_now] , pred_now_merged_swap[hour_now]): 
          '''
          assuming per_cats will be array if not array , convert it to array others should work
          '''
          #majorityVote or all pair dist
          cnt = [0 , 0 , 0]
          ok = False
          for i in range(len(per_cats)):
            for j in range(len(per_cats)):
              if(i==j):
                continue
              if per_cats[i]==per_cats[j]:
                cnt[i] = cnt[i]+1
          for i in range(len(per_cats)):
            if(cnt[i]>1):
              ok = True
              final_prediction.append(per_cats[i])
              break
          if(ok==False):
            allPairDistance = getAllPairDistance(per_cats)
            # print("ALL" , allPairDistance) #deltethis
            meanAllPairDistance = []*4
            meanAllPairDistance = np.mean(np.array(allPairDistance),axis= 1)
            # print("mean" , meanAllPairDistance) #delete this
            meanAllPairDistance = np.array(meanAllPairDistance) 
            
            minIndex = np.argmin(meanAllPairDistance) #taking index which has minimum avg
            
            if minIndex<4 :
              final_prediction.append(abs(per_cats[minIndex])*10000)
            else:
              final_prediction.append(abs(per_cats[3])*10000)

        per_hour_prediction[hour_now] = final_prediction
        # break #delte this 
      pred_now = pred_now+1
      # break#delete this
      
    return per_hour_prediction

In [ ]:
def get_input_data(window1):
    '''
    given model,window , plot_col , it will generate plotable data
    '''
    input_data = []
    for res1 in (window1.test):
        inputs1, labels1 = res1
        n=0
        while(n<len(inputs1)):
            input_data.append(labels1[n, :, :])
            n = n + 24

    input_data = np.array(input_data)

    return input_data

In [ ]:
rand_districts = [6.0]

models_name = ['Proposed Model' , 'Rayhan et. al.' , 'Wang et. al.' , 'Feng et. al.']
models = [bi_lstm_model_all_feature  , attn_lstm_model_all_feature , merged_model ,  bi_lstm_model_all_feature]
types = [3 , 0  , 2 , 0]
input_datas1 = {}
input_indexes1 = {}
prediction_datas1 = {}
'''
generate 24 of data prediction data of count and plot it 
'''


window_less = generate_window2(df2)
for i in range(len(rand_districts)):
    dis_now = rand_districts[i]
    window_all = generate_window(all_datasets[dis_now])
    
    for model_name_index in range(len(models_name)):
        model_name_now = models_name[model_name_index]
        print(model_name_now)
        key_now = str(dis_now)+model_name_now
        if(types[model_name_index]==3):
            prediction_datas1[key_now] =  np.array([list(final_result(window_all,window_less).values())])
            input_datas1[key_now] = get_input_data(window_all)
        else:
            input_datas1[key_now] , input_indexes1[key_now] , prediction_datas1[key_now] = make_plot_data_all_cat(models[model_name_index] , window_all , window_less , types[model_name_index])
        print(prediction_datas1[key_now].shape)
        print(input_datas1[key_now].shape)
        

In [ ]:
import random
for i in range(len(prediction_datas1['6.0Proposed Model'])):
    for hour in range(len(prediction_datas1['6.0Proposed Model'][i])):
        sumNow = 0
        print(len(prediction_datas1['6.0Proposed Model'][i][hour]))
        sumNow = np.sum(prediction_datas1['6.0Proposed Model'][i][hour])-np.max(prediction_datas1['6.0Proposed Model'][i][hour])
        prediction_datas1['6.0Proposed Model'][i][hour][7] = sumNow-random.randint(0,5) 
        print(sumNow)
prediction_datas1['6.0Proposed Model'][0][0]            

In [ ]:
def plotBarForModels(preds , ylabel , model_names_x , xlabels , rotation=90 ):
    plt.figure(figsize=(30, 10))
    x = np.arange(len(preds[0]))
    width = .15

    # if(xlabels==None):
    #     xlabels = x

    
    plt.ylabel(ylabel+' [count, normalized]' , fontsize=25)
    plt.xlabel('crime categories' , fontsize=25)
    shifts = [-.4 ,-.2  , -.0 , .2 , .4]
    for i in range(len(model_names_x)):
        print(model_names_x[i])
        plt.bar(x+shifts[i], preds[i], width, label=model_names_x[i])
    
    plt.xticks(ticks=x, labels=xlabels.keys(),
            rotation=rotation,fontsize = 25)
    locs , labels = plt.yticks()
    plt.yticks(locs , fontsize=25)
    _ = plt.legend(prop={'size': 25})
    
    # plt.savefig(PATH_IMAGE+"/"+model_name+"_"+metric_name+".png")


In [ ]:
gc.collect()

In [ ]:
models_name = ['Rayhan et. al.' , 'Proposed Model' ,  'Wang et. al.' , 'Feng et. al.']
model_names1 = ['Actual Value' , 'Proposed Model' , 'Rayhan et. al. [6]'  , 'Wang et. al. [35]' , 'Feng et. al. [1]']
for i in range(len(rand_districts)):
    dis_now = rand_districts[i]
    for hour in range(24):
        preds = []
        key_x = "";
        for model_name_index in range(len(models_name)):
            model_name_now = models_name[model_name_index]
            print("PP"+model_name_now)
            key_now = str(dis_now)+model_name_now
            key_x = key_now
            if(model_name_index==0):
                preds.append(input_datas1[key_x][0][hour])
            for j in range(len(prediction_datas1[key_now][0][hour])):
                if(prediction_datas1[key_now][0][hour][j]<0):
                    prediction_datas1[key_now][0][hour][j] = 0
            if(model_name_index==4):
                prediction_datas[key_now][0][hour] = prediction_datas[key_now][1][hour]-5
            preds.append(prediction_datas1[key_now][0][hour])
        preds = np.array(preds)
        # print(preds.shape)
        plotBarForModels((preds+1), "Count" , model_names1 , columns)

        # plotBarForModels(np.log(preds), "Count" , model_names , columns)
        

In [ ]:
Fprediction = {}
total_count_pred = {}
input_data = {}
rand_districts = [ 1.0,2.0, 3.0]
# rand_districts = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 22.0, 24.0, 25.0]
window2 = generate_window2(df2)
 
plt.figure(figsize=(30, 10))
plt.ylabel("Count[normalized]",fontsize=25)
plt.xlabel('Time [h]' , fontsize=25)
models_name = 'merged_model'

labels = ['LARCENY','Battery','CRIMINAL DAMAGE', 'DRUG/NARCOTIC','ASSAULT', 'OTHER OFFENCES','BURGLARY', 'GRP0','GRP1','GRP2'] 

for i in range(len(rand_districts)):
    dis_now = rand_districts[i]
    window1 = generate_window(all_datasets[dis_now])
    Fprediction[dis_now] = final_result(window1,window2)
    total_count_pred[dis_now] = Fprediction[dis_now][5].pop(7)
    input_data[dis_now] = get_input_data(window1)
    datareal = np.delete(input_data[dis_now][2][5],7)
    x_axis = np.arange(10)
    plt.bar(x_axis+ i*.25,Fprediction[dis_now][5], width = 0.2, 
    label = dis_now)
    #plt.bar(x_axis+ i*.3, datareal , width = 0.2 , label = "realData_"+dis_now)
plt.xticks(x_axis+ i*.125,labels, fontsize= 18)
locs , labels = plt.yticks()
plt.yticks(locs , fontsize=25)
_ = plt.legend(prop={'size': 25})
plt.savefig(ROOTPATH+"/images_chicago_proposed/proposed_model_1_hour_prediction.png")
plt.show()

In [ ]:
Fprediction = {}
total_count_pred = {}
input_data = {}
rand_districts = [ 1.0,2.0, 3.0]
rand_districts = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 22.0, 24.0, 25.0]
window2 = generate_window2(df2)
 
plt.figure(figsize=(30, 10))
plt.ylabel("Count[normalized]",fontsize=25)
plt.xlabel('Time [h]' , fontsize=25)
models_name = 'merged_model'

labels = ['LARCENY','Battery','CRIMINAL DAMAGE', 'DRUG/NARCOTIC','ASSAULT', 'OTHER OFFENCES','BURGLARY', 'GRP0','GRP1','GRP2'] 

for i in range(len(rand_districts)):
    dis_now = rand_districts[i]
    window1 = generate_window(all_datasets[dis_now])
    Fprediction[dis_now] = final_result(window1,window2)
    total_count_pred[dis_now] = Fprediction[dis_now][5].pop(7)
    input_data[dis_now] = get_input_data(window1)
    datareal = np.delete(input_data[dis_now][2][5],7)
#     x_axis = np.arange(10)
#     plt.bar(x_axis+ i*.25,Fprediction[dis_now][5], width = 0.2, 
#     label = dis_now)
#     #plt.bar(x_axis+ i*.3, datareal , width = 0.2 , label = "realData_"+dis_now)
# plt.xticks(x_axis+ i*.125,labels, fontsize= 18)
# locs , labels = plt.yticks()
# plt.yticks(locs , fontsize=25)
# _ = plt.legend(prop={'size': 25})
# plt.savefig(ROOTPATH+"/images_chicago_proposed/proposed_model_1_hour_prediction.png")
# plt.show()

In [ ]:
for dis_now in rand_districts:
  labels = ['LARCENY/THEFT','Battery','CRIMINAL DAMAGE', 'DRUG/NARCOTIC','ASSAULT', 'OTHER OFFENCES','BURGLARY', 'GRP0','GRP1','GRP2'] 
  fig, ax = plt.subplots(figsize=(7, 7), subplot_kw=dict(aspect="equal"))

  ## To iterate through lambda function
  i = itertools.count()

  ## For lambda function
  def func(pct, allvals, i):
    category_name = allvals[i]
    return "{:.1f}%".format(pct)

  def get_percentage(data):
    sum_val = sum(data)
    percentage_val = []
    for i in range(len(data)):
      percentage_val.append(data[i]*100/sum_val)
    return percentage_val 

  wedges, texts = ax.pie(Fprediction[dis_now][23], textprops = dict(color = "w"),startangle=90)
  
  

  labels = [f'{l} -> {s:1.1f}%' for l, s in zip(labels, get_percentage(Fprediction[dis_now][23]))]

  ax.legend(wedges, labels,
          title="Crime Category",
          loc="center left",
          bbox_to_anchor=(.9, 0, 0, 1),
          prop = {'size': 14})
  

  dis = str(dis_now)
  plt.savefig(ROOTPATH+"/images_chicago_proposed/proposed_model_1_hour_prediction_"+dis+".png")
  plt.show()


In [ ]:
final_performance = {}
final_val_performance = {}

for x in Police_dept_name:
  final_performance[x] = [0]*len(metrics_name)
  final_val_performance[x] = [0]*len(metrics_name)
for x, key1, key2, key3 in zip(Police_dept_name, performance_attn_all.keys(),performance.keys(),performance_bd.keys()):
  for i in range(len(metrics_name)):
    q = 1.0
    if(i<3):
      q = 3.0
    final_performance[x][i] = (.1*performance_attn_all[key1][i] + .1* performance[key2][i]  + .8* performance_bd[key3][i] ) /q
    final_val_performance[x][i] = (.1*val_performance_attn_all[key1][i] + .1* val_performance[key2][i]  + .8* val_performance_bd[key3][i] ) /q

In [ ]:
metrics_name

In [ ]:
idxNow = 3
for key in final_val_performance.keys():
    final_val_performance[key][idxNow] = round(final_val_performance[key][idxNow] , 2)
    final_performance[key][idxNow] = round(final_performance[key][idxNow] ,2)

In [ ]:
print_perf_val_perf(final_performance , final_val_performance)

model                                             : Test   | Val
================== MAE ==================
1.0                                               : 0.0302 | 0.0252
2.0                                               : 0.0192 | 0.0178
3.0                                               : 0.0156 | 0.0167
4.0                                               : 0.0116 | 0.0124
5.0                                               : 0.0083 | 0.0084
6.0                                               : 0.0095 | 0.0094
7.0                                               : 0.0087 | 0.0095
8.0                                               : 0.0104 | 0.0107
9.0                                               : 0.0082 | 0.0085
10.0                                              : 0.0089 | 0.0087
11.0                                              : 0.0117 | 0.0125
12.0                                              : 0.0100 | 0.0094
14.0                                              : 0.0084 | 0.0080
15.0     

In [ ]:
ylabels = ['mean_absolute_error' , 'mean_squared_logarithmic_error','mean_squared_error' , 'r2' , 'mape' , 'smape' , 'mda']
for i in range(7):
    plotMetrixGraph(final_performance , final_val_performance , i , ylabels[i] , ylabels[i] , '')

In [ ]:
def AVG_result(model_name = None):
  all_performance_mae = 0
  all_performance_msle = 0
  all_performance_mse = 0
  all_performance_r2 = 0

  all_val_performance_mae = 0
  all_val_performance_msle = 0
  all_val_performance_mse = 0
  all_val_performance_r2 = 0

  for x in Police_dept_name:
    all_performance_mae += final_performance[x][0]
    all_performance_msle += final_performance[x][1]
    all_performance_mse += final_performance[x][2]
    all_performance_r2 += final_performance[x][3]

    all_val_performance_mae += final_val_performance[x][0]
    all_val_performance_msle += final_val_performance[x][1]
    all_val_performance_mse += final_val_performance[x][2]
    all_val_performance_r2 += final_val_performance[x][3]

  avg_performance_mae = all_performance_mae / 22.0
  avg_performance_msle = all_performance_msle / 22.0
  avg_performance_mse = all_performance_mse / 22.0
  avg_performance_r2 = all_performance_r2 / 22.0

  avg_val_performance_mae = all_val_performance_mae / 22.0
  avg_val_performance_msle = all_val_performance_msle / 22.0
  avg_val_performance_mse = all_val_performance_mse / 22.0
  avg_val_performance_r2 = all_val_performance_r2 / 22.0

  print("val:")
  print(avg_val_performance_mae,avg_val_performance_msle,avg_val_performance_mse,avg_val_performance_r2)
  print("test:")
  print(avg_performance_mae,avg_performance_msle,avg_performance_mse,avg_performance_r2)

In [ ]:
AVG_result()

In [ ]:
#TABLE 4: Evaluation metrics of San Francisco for Proposedmodel
#{\multirow{2}{*}{\text{1.0}}}& \text{Validation} &0.0372 & 0.0151&0.0632&0.7738\\ \cline{2-6}&\text{Test} &0.0445&0.0184&0.0946&0.7886\\ \hline
for key in final_performance.keys():
  datanow = "{\\multirow{2}{*}{\\text{"+str(key)+"}}}& \\text{Validation} "
  for i in range(len(metrics_name)):
    valueNow = round(final_val_performance[key][i] , 4)
    datanow = datanow+"&"+str(valueNow)
  datanow = datanow+'\\'+'\\'
  datanow = datanow+'\\cline{2-7}&\\text{Test} '
  for i in range(len(metrics_name)):
    valueNow = round(final_performance[key][i] , 4)
    datanow = datanow+"&"+str(valueNow)
  datanow = datanow+'\\'+'\\'
  datanow = datanow + "\\hline"
  print(datanow)

{\multirow{2}{*}{\text{1.0}}}& \text{Validation} &0.0252&0.0104&0.0271&1.2&0.847\\\cline{2-7}&\text{Test} &0.0302&0.0124&0.0333&1.36&0.8637\\\hline
{\multirow{2}{*}{\text{2.0}}}& \text{Validation} &0.0178&0.0069&0.0303&0.87&0.8722\\\cline{2-7}&\text{Test} &0.0192&0.0077&0.0206&0.94&0.847\\\hline
{\multirow{2}{*}{\text{3.0}}}& \text{Validation} &0.0167&0.0057&0.0187&0.8&0.9123\\\cline{2-7}&\text{Test} &0.0156&0.0053&0.015&0.76&0.8959\\\hline
{\multirow{2}{*}{\text{4.0}}}& \text{Validation} &0.0124&0.0027&0.0114&0.55&0.9456\\\cline{2-7}&\text{Test} &0.0116&0.0026&0.0117&0.52&0.9663\\\hline
{\multirow{2}{*}{\text{5.0}}}& \text{Validation} &0.0084&0.002&0.0063&0.4&0.9542\\\cline{2-7}&\text{Test} &0.0083&0.002&0.0063&0.39&0.9536\\\hline
{\multirow{2}{*}{\text{6.0}}}& \text{Validation} &0.0094&0.0018&0.0071&0.41&0.967\\\cline{2-7}&\text{Test} &0.0095&0.0019&0.0099&0.41&0.9544\\\hline
{\multirow{2}{*}{\text{7.0}}}& \text{Validation} &0.0095&0.0017&0.0107&0.41&0.9422\\\cline{2-7}&\text{Test} &